# TME 6 : Projet Detection de motifs


## Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons implémenter deux algorithmes randomisés pour la recherche de motifs : Greedy Profile Motif Search et random Projections.


1\. Nous allons réutiliser les fonctions du TME5 pour générer `t` séquences artificielles de taille `n`, et implanter dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement.

In [ ]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence


In [ ]:
def generateRandomSequence(n, upper=True):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront majuscule, False minuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    nucList = [nuc[random.randint(0,3)].upper() if upper else nuc[random.randint(0,3)].lower() for i in range(n)]
    sequence = "".join(nucList)
    return sequence

def modifierMotif(motif, nbpos,  upper=True):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront majuscule, False minuscule
    sortie motifM: motif modifié
    """
    motifM = motif
    posToModified = random.sample(range(len(motif)), nbpos)
    for pos in posToModified:
        newNuc = random.choice(nuc)
        while newNuc == motif[pos]:
            newNuc = random.choice(nuc)
        motifM = motifM[0:pos] + newNuc + motifM[pos+1:]
    return motifM

def implantMotifVar(k, v, t, n):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus le motif doit être égal à n, pensez à générer de séquence aléatoire de taille n-k pour pouvoir implanter un motif de taille k
    """
    motif = generateRandomSequence(k)
    print("motif: ", motif)
    modifiedMotifs = [modifierMotif(motif, v) for i in range(t)]
    print("modifiedMotifs: ", modifiedMotifs)
    posToInsert = [random.randint(0, n-k-1) for i in range(t)]
    preSequences = [generateRandomSequence(n-k) for i in range(t)]
    sequences = [preSequences[i][:posToInsert[i]+1] + modifiedMotifs[i] + preSequences[i][posToInsert[i]+1:] for i in range(t)]
    return sequences

adn = implantMotifVar(k, v, t, n)
print (adn)

adn  = [s.upper() for s in adn]
print (adn)

Réponse:

motif:  GATAT
modifiedMotifs:  ['GATAC', 'GATAG', 'GCTAT']
['GATGATACAT', 'CTAAGATAGT', 'TCGCTATCAA']
['GATGATACAT', 'CTAAGATAGT', 'TCGCTATCAA']

2\. Faire une fonction pour sélectionner des positions de départ aléatoirement `s = (s1, …,st)`.


In [ ]:
#creating vector s
#Get t random positions from 0 to n - k
def generateRandomS(sequences, k):
	"""
	Génère un vecteur de position aléatoires
	entrée sequences: matrice de dimension txn contenant les sequences
	entrée k: taille du motif
	sortie s: vecteur de position aléatoires, une position par séquence
  REMARQUE les positions doivent être inférieur à n-k
	"""
	return [random.randint(0, len(sequences[i])-k) for i in range(len(sequences))]

s = generateRandomS(adn, k)
print (adn)
print (s)

Réponse:

['AAATGTTAGA', 'GTATTTACCC', 'GACACTTACG']
[0, 5, 3]

['ATTAGCATTT', 'CTTAGTGGAT', 'ATTATAGCCA']

[0, 0, 0]


3\. Extraire les motifs en utilisant le vecteur `s`, et construire un profile (matrice de fréquence). Attention, utiliser les pseudocount 1 pour éviter les plus tard les probabilités à zéro.


In [ ]:
def extractSeqs(s, seqs, k):
    """
    Extraire les motifs des séquences à l'aide de positions s
    entrée s: vecteur contenant les positions de départs
    entrée seqs: matrice de dimension txn avec les séquences
    entrée k: taille du motif
    sortie motifs: liste de motifs
    """
    motifs = []
    try:
        motifs = [seqs[i][s[i]:s[i]+k] for i in range(len(seqs))]
    except IndexError:
        pass
    return motifs

motifs = extractSeqs(s, adn, k)
 print("motifes",motifs)

#Construire matrice de fréquence
def profile(motifs, k, nuc):
    """
    Construire une matrice de fréquence de dimension k x |nuc|
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie MF: matrice de fréquence
    """
    nDict = {nuc[i]: i for i in range(len(nuc))}
    q = len(nuc)
    mf = np.ones((q, k))
    for col in range(k):
        for line in range(len(motifs)):
            # try:
            nucleotide = motifs[line][col]
            # except IndexError:
                # print(line, col, motifs)
            mf[nDict[nucleotide]][col] += 1
    return mf
    
MF = profile(motifs, k, nuc)
print("MF: ",MF)

Réponse:

motifs:  ['ACGTC', 'GACAA', 'CATAG']
MF:  [[2. 3. 1. 3. 2.]
 [2. 2. 2. 1. 2.]
 [2. 1. 2. 1. 2.]
 [1. 1. 2. 2. 1.]]s

4\. Transformer la matrice de fréquence en PWM. Il faut diviser chaque élément par la somme de ses colonnes. Faire aussi une fonction pour calculer la sequence consensus d'une matrice de frequence.

In [ ]:
# motifs = [s.upper() for s in motifs]

def generatePWM(MF):
  """
  Transforme la matrice de fréquence en PWM
  entrée MF: matrice de fréquence
  sortie PWM: matrice de probabilité (poids positions)
  """
  q = len(nuc)
  colSumList = np.sum(MF, 0)
  colSum = colSumList[0]
  for i in range(1, len(colSumList)):
      assert(colSumList[i] == colSum)
  # MF = np.ones((q, k))
  # print(PWM)
  return np.array([MF[i, j]/colSum for i in range(q) for j in range(k)]).reshape(q, k)

PWM = generatePWM(MF)
print("PWM: ",PWM)

Réponse:

PWM:  [[0.14285714 0.28571429 0.28571429 0.14285714 0.14285714]
 [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714]
 [0.28571429 0.28571429 0.28571429 0.28571429 0.14285714]
 [0.42857143 0.28571429 0.28571429 0.42857143 0.57142857]]

In [ ]:
def getConsensus(MF, k, nuc):
    """
    Renvoie la sequence consensus d'une matrice de frequence
    entrée MF: matrice de fréquence
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie seqCons: sequence consensus
    N.B. en cas d'égalité, on prend la sequence avec les premières nucléotides selon l'ordre de ACGT
    """
    minIndexArray = np.argmax(MF, axis=0)
    return "".join([nuc[index] for index in minIndexArray])

seqCons = getConsensus(MF, k, nuc)
print("sequence consensus: ",seqCons)

Réponse: Sequence consensus:  GAGTG

5\. Faire une fonction pour calculer la probabilité d'un motif de taille `k` selon une PWM.


In [ ]:
def probability(PWM, motif):
    """
    Calcul la probalité d'un motif selon PWM
    entrée PWM: matrice de probabilité (poids positions)
    entrée motif: motif
    sortie prob: probalité Prob(motif|PWM)
    """
    pos = 0; prob = 1
    #print (PWM)
    for n in motif:
        i = nuc.index(n)
        #print (i, pos)
        prob *=PWM[i, pos]
        pos +=1
    return prob

prob = probability(PWM, seqCons)
print (prob)

Réponse:

La probabilité du motif selon la PWM est 0.006425893972749447

6\. Faire une fonction pour calculer le pMostProbkmer d'une séquence, voir un exemple dans les slides de cours.

In [ ]:
def pMostProbkmer(P, k, sequence):
    """
    calcul la position du k-mer le plus probable dans la séquence
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    sortie s: la position du k-mer le plus probable dans la sequence
    """
    probList = [probability(P, sequence[i:i+k]) for i in range(len(sequence)-k+1)]
    maxList = max(probList) # le premier plus grand
    return probList.index(maxList)


s = pMostProbkmer(PWM, k, "GCC" + seqCons+ "GGATACT")
print ("La position du k-mer ",s)

Réponse:

La position du k-mer : 3

5


7\. Faire une fonction pour obtenir les nouvelles positions de départ `s = (s1, …,st)`, c’était à dire les positions qui contiens les k-mer le plus probables.

In [ ]:
def getNewS(PWM, k, sequences):
    """
    Trouve les nouvelles positions des k-mer le plus probables
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    sortie s: vecteur avec les positions les plus problables
    """
    s = [pMostProbkmer(PWM, k, seq) for seq in sequences]
   
    return s

ns = getNewS(PWM, k, adn)
print ("Les nouvelles positions des k-mer lees plus probables",ns)

Réponse:

Les nouvelles positions des k-mer lees plus probables [1, 1, 1]

8\. La condition d’arrêt de l'algorithme est le non changement de la matrice de fréquence d'une itération à l'autre, 
faire une fonction pour comparer deux matrices et détecter le changement. 
Faire aussi une fonction pour obtenir le score d'une matrice de fréquence : la somme de max de chaque colonne

In [ ]:
def changeProfile(P1, P2):
    """
    Compare deux matrice
    entrée P1: matrice de fréquence
    entrée P2: matrice de fréquence
    sortie: True si les matrices sont différents, False au contraire
    """
    print(P1, P2, np.mat(P1) == np.mat(P2), (np.mat(P1) == np.mat(P2)).all())
    return (np.mat(P1) == np.mat(P2)).all()

def getScore(MF, k):
    """
    Renvoie le score de MF, la somme des max de chaque colonne
    entrée P: matrice de fréquence
    entrée k: taille du motif
    sortie sc: score
    """
    mf = np.mat(MF)
    maxArray = np.max(mf, axis=0)
    return np.sum(maxArray)

# print(adn)
# print(seqCons)

print("La MF: ",MF)
sc = getScore(MF, k)
print ("Le score de MF: ",sc)

Réponse:

La MF:  [[1. 2. 2. 1. 3.]
 [2. 2. 1. 2. 2.]
 [1. 2. 3. 3. 1.]
 [3. 1. 1. 1. 1.]]
Le score de MF:  14.0

9\. Implementer l'algorithme ``GreedyProfileMotifSearch`` en utilisant les fonctions precedentes. 


In [ ]:
def GreedyProfileMotifSearch(sequences, t, n, k):
    """
    Implémente l'algorithme GreedyProfileMotifSearch 
	  entrée sequences: matrice de dimension txn contenant les séquences 
	  entrée t : nombre de séquences 
	  entrée n : longueur des séquences 
	  entrée k : taille du motif 
    sortie s : vecteur de positions de départ ayant le meilleur motif 
    sortie bestScore : le score associé à s
    """
    s = generateRandomS(sequences, k)
    bestScore = 0
    bestS = []
    while getScore(profile(extractSeqs(s, sequences, k), k, nuc), k) > bestScore:
        bestS = s
        tmpProfile = profile(extractSeqs(s, sequences, k), k, nuc)
        bestScore = getScore(tmpProfile, k)
        s = getNewS(tmpProfile, k, sequences)
    return bestS, int(bestScore)


s, bestScore = GreedyProfileMotifSearch(adn, t, n, k)   
motifs = extractSeqs(s, adn, k)
print ("Les motifs: ",motifs)

Réponse:

Les motifs:  ['CCGGG', 'GAGGC', 'CAGGA']

10\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour le trouver. 

Reponse:

In [ ]:
nuc = ('A', 'C', 'G', 'T')

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence

mv = implantMotifVar(k, v, t, n)
print("sequences:", mv)

sMv, bestScoreMv = GreedyProfileMotifSearch(mv, t, n, k)
print("s, bestScore:", sMv, bestScoreMv)
seqsMv = extractSeqs(sMv, mv, k)
print(seqsMv)
consensusMv = getConsensus(profile(seqsMv, k, nuc), k, nuc)
print("Le consensusMv: ",consensusMv)

Réponse:

sequences: ['CCAACTCAAC', 'ACTCTCACGT', 'TCACCTGAGG']
s, bestScore: [3, 2, 0] 17
['ACTCA', 'TCTCA', 'TCACC']
Le consensusMv:  TCTCA

11\. Vous avez certainement observer que l’algorithme ne produire pas toujours la même sortie et que les résultats dépendent de la sélection aléatoire des positions de départ `s`. Pour augmenter nous chances de retrouvez les bons motifs, nous allons implémenter une version itérative ``GreedyProfileMotifSearchIte`` 
qui a chaque fois sauvegarde `s` et le score du profile associé à `s`, après `I` itération, l’algorithme renvoie le vecteur `s` ayant le plus grand score.

In [ ]:
def GreedyProfileMotifSearchIte(sequences, t, n, k, It=50):
    """
    Implémente l'algorithme GreedyProfileMotifSearch iteratif
	  entrée séquences: matrice de dimension txn contenant les séquences 
	  entrée t : nombre de séquences 
	  entrée n : longueur des séquences 
	  entrée k: taille du motif 
    entrée It: nombre d'iterations
    sortie positions: dictionnaire clé=score, valeur= vecteur s 
    sortie consensus: dictionnaire clé=score, valeur= consensus sequence du motif
    """
    pos = {}
    consensus = {}
    for i in range(It):
        s, Score = GreedyProfileMotifSearch(sequences, t, n, k)
        motifConsensus = getConsensus(profile(extractSeqs(s, sequences, k), k, nuc), k, nuc)
        if Score not in pos.keys():
            pos[Score] = s
            consensus[Score] = motifConsensus
    return pos, consensus

def sortSCK(posDict, consensusDict, rev = True):
    return sorted([(posDict[k], consensusDict[k],k) for k in posDict.keys()], key=lambda item:item[2], reverse = rev)

s, consensus = GreedyProfileMotifSearchIte(adn, t, n, k, 50)
print(s, consensus)

Réponse:

{14: [1, 3, 5], 15: [2, 1, 2], 13: [5, 3, 2], 17: [5, 4, 0], 12: [0, 4, 2], 16: [5, 4, 4]} {14: 'CGCAT', 15: 'GAATG', 13: 'TAATT', 17: 'TGTTT', 12: 'TCACA', 16: 'TCTTT'}

In [ ]:
nuc = ('A', 'C', 'G', 'T')

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence

mv2 = implantMotifVar(k, v, t, n)
print("sequences:", mv2)

posDict, consensusDict = GreedyProfileMotifSearchIte(mv2, t, n, k)


print("s, concensus, k: ", sortSCK(posDict, consensusDict))

Réponse:

sequences: ['CGTCCAGCTT', 'GTCCCAATGG', 'CTATTCCGGC']
s, concensus, k:  [([1, 0, 2], 'GTCCC', 17), ([3, 3, 0], 'CCAAT', 16), ([1, 0, 0], 'GTCCA', 15), ([1, 4, 0], 'CTATA', 14), ([5, 5, 3], 'AACCG', 13), ([5, 3, 1], 'AAATT', 12)]

12\. Tester algorithme  ``GreedyProfileMotifSearchIte`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats.

In [ ]:
k=7; v=1; t=20; n=80

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "Sequence_by_Peaks_5.fasta" #Remplacer par votre fichier

sequencesChip = readFasta(genome, n)
t = len(sequencesChip)
s, consensus = GreedyProfileMotifSearchIte(sequencesChip, t, n, k)
print(s, concensus)

Répinse:


{2026: [13, 33, 72, 48, 27, 44, 7, 64, 23, 65, 45, 15, 35, 8, 22, 26, 26, 55, 16, 70, 28, 35, 12, 30, 43, 73, 43, 41, 45, 17, 17, 21, 20, 55, 24, 0, 55, 48, 56, 69, 0, 16, 18, 1, 47, 10, 47, 7, 64, 65, 47, 24, 38, 54, 6, 49, 42, 46, 64, 14, 25, 2, 27, 17, 30, 60, 66, 67, 38, 69, 18, 36, 66, 53, 39, 28, 71, 1, 73, 39, 57, 29, 12, 7, 65, 30, 57, 53, 14, 19, 5, 56, 64, 73, 25, 48, 29, 40, 46, 53, 1, 15, 16, 16, 67, 57, 19, 7, 20, 66, 72, 62, 70, 12, 54, 14, 38, 14, 18, 69, 22, 24, 71, 9, 73, 43, 12, 50, 33, 21, 36, 68, 20, 67, 16, 22, 11, 71, 26, 43, 55, 14, 62, 33, 73, 18, 29, 20, 53, 9, 50, 8, 68, 69, 40, 17, 69, 68, 11, 22, 23, 22, 62, 67, 19, 67, 51, 1, 5, 67, 72, 35, 60, 51, 5, 65, 17, 19, 16, 41, 36, 63, 17, 57, 60, 23, 11, 61, 6, 51, 10, 3, 72, 26, 59, 55, 13, 69, 24, 7, 40, 2, 11, 37, 42, 62, 51, 1, 39, 43, 20, 27, 37, 55, 34, 56, 29, 58, 20, 55, 12, 51, 9, 32, 52, 2, 22, 13, 19, 38, 4, 62, 49, 4, 36, 36, 56, 10, 69, 53, 13, 27, 70, 65, 35, 57, 15, 47, 37, 64, 54, 29, 65, 7, 17, 28, 47, 49, 9, 18, 37, 5, 32, 67, 62, 2, 55, 73, 73, 15, 59, 0, 23, 40, 66, 37, 26, 54, 13, 66, 73, 55, 8, 66, 3, 40, 58, 23, 34, 10, 30, 37, 30, 68, 70, 72, 34, 5, 39, 4, 30, 63, 52, 47, 52, 37, 65, 34, 21, 19, 67, 3, 17, 72, 1, 32, 55, 1, 21, 28, 70, 18, 23, 56, 45, 70, 33, 53, 65, 56, 5, 65, 64, 64, 10, 7, 7, 63, 33, 34, 52, 62, 1, 70, 11, 69, 54, 7, 13, 9, 60, 26, 11, 52, 53, 73, 16, 7, 33, 33, 1, 13, 14, 33, 38, 30, 21, 62, 21, 6, 24, 32, 51, 73, 55, 58, 68, 65, 18, 45, 59, 6, 65, 16, 7, 6, 35, 12, 9, 63, 19, 45, 66, 71, 70, 62, 7, 5, 22, 51, 63, 55, 72, 37, 41, 30, 64, 67, 72, 9, 22, 10, 15, 68, 33, 7, 19, 24, 69, 21, 53, 3, 46, 26, 35, 8, 23, 4, 72, 21, 5, 67, 37, 41, 27, 36, 35, 70, 42, 24, 61, 13, 30, 22, 53, 36, 21, 35, 20, 9, 37, 51, 11, 18, 69, 68, 54, 59, 30, 24, 66, 29, 41, 72, 8, 57, 18, 62, 7, 18, 33, 70, 69, 12, 1, 58, 23, 65, 26, 55, 67, 33, 70, 27, 17], 2059: [14, 71, 47, 13, 29, 32, 12, 64, 7, 67, 45, 16, 52, 18, 25, 44, 26, 1, 20, 72, 29, 44, 3, 27, 44, 19, 25, 39, 71, 24, 58, 25, 28, 24, 26, 45, 62, 48, 57, 70, 47, 18, 17, 2, 47, 18, 49, 73, 53, 66, 45, 54, 39, 21, 21, 23, 66, 65, 64, 49, 13, 25, 26, 69, 73, 9, 58, 21, 37, 68, 19, 36, 64, 53, 36, 29, 71, 56, 37, 60, 47, 32, 11, 38, 47, 30, 69, 55, 21, 52, 64, 41, 7, 52, 25, 47, 20, 73, 49, 72, 72, 65, 4, 69, 68, 61, 17, 67, 20, 67, 21, 51, 71, 12, 44, 40, 39, 65, 26, 40, 21, 17, 26, 0, 52, 0, 10, 3, 50, 39, 63, 69, 52, 20, 61, 12, 61, 68, 24, 3, 59, 13, 64, 34, 65, 17, 55, 20, 11, 34, 51, 7, 67, 16, 61, 66, 70, 67, 13, 30, 38, 19, 63, 16, 45, 47, 52, 69, 20, 29, 32, 70, 39, 43, 5, 39, 16, 19, 17, 43, 37, 45, 35, 45, 65, 63, 62, 61, 10, 52, 67, 50, 41, 27, 8, 23, 15, 70, 39, 27, 39, 72, 12, 59, 40, 11, 59, 1, 40, 39, 9, 43, 38, 59, 34, 52, 15, 59, 21, 66, 62, 50, 51, 31, 50, 48, 24, 13, 20, 65, 3, 21, 50, 5, 38, 37, 63, 10, 33, 54, 12, 29, 7, 10, 36, 33, 3, 72, 37, 65, 55, 72, 64, 2, 18, 18, 70, 11, 12, 32, 70, 3, 32, 20, 26, 3, 55, 5, 70, 19, 42, 1, 38, 38, 66, 71, 3, 50, 53, 37, 37, 23, 15, 66, 2, 49, 40, 73, 35, 70, 27, 60, 35, 70, 9, 67, 34, 0, 40, 3, 73, 1, 54, 46, 51, 38, 65, 35, 20, 20, 64, 66, 18, 73, 2, 61, 60, 2, 6, 11, 69, 3, 71, 54, 44, 70, 31, 69, 56, 55, 61, 67, 27, 24, 11, 67, 7, 61, 46, 51, 35, 0, 3, 38, 48, 73, 56, 26, 27, 61, 59, 53, 29, 33, 59, 72, 16, 12, 17, 34, 2, 50, 61, 35, 35, 32, 2, 62, 61, 72, 35, 57, 59, 72, 56, 18, 68, 61, 62, 21, 60, 28, 4, 34, 8, 31, 33, 6, 6, 58, 21, 66, 66, 16, 16, 63, 48, 58, 22, 52, 8, 52, 32, 42, 72, 22, 40, 57, 73, 8, 21, 10, 71, 56, 30, 8, 40, 64, 28, 20, 9, 53, 31, 25, 37, 45, 59, 26, 71, 23, 6, 37, 66, 57, 59, 39, 54, 71, 41, 31, 61, 11, 29, 62, 36, 26, 20, 55, 48, 21, 26, 50, 8, 19, 8, 27, 55, 8, 3, 52, 72, 30, 5, 71, 6, 17, 68, 6, 6, 16, 33, 29, 59, 13, 61, 32, 23, 64, 26, 57, 69, 12, 56, 19, 18], 1991: [9, 72, 49, 61, 28, 62, 7, 58, 33, 65, 39, 15, 23, 18, 19, 50, 25, 30, 53, 13, 73, 41, 19, 29, 41, 5, 46, 54, 71, 16, 66, 51, 51, 19, 26, 47, 59, 48, 58, 68, 47, 16, 18, 0, 47, 12, 3, 73, 63, 67, 43, 38, 37, 21, 5, 51, 42, 50, 64, 51, 47, 26, 27, 19, 6, 32, 57, 70, 37, 69, 67, 73, 65, 53, 2, 28, 65, 0, 73, 38, 55, 29, 55, 6, 31, 29, 56, 52, 26, 19, 5, 44, 63, 68, 64, 47, 23, 53, 49, 52, 71, 15, 4, 73, 66, 61, 19, 66, 62, 52, 60, 61, 69, 20, 43, 3, 40, 27, 52, 26, 9, 18, 71, 20, 73, 42, 11, 47, 40, 48, 28, 68, 57, 66, 15, 23, 10, 37, 24, 2, 52, 10, 13, 54, 72, 17, 29, 21, 49, 47, 63, 7, 55, 15, 39, 29, 18, 64, 14, 32, 23, 26, 62, 67, 68, 51, 54, 70, 5, 66, 72, 35, 41, 65, 41, 42, 16, 21, 55, 44, 35, 63, 17, 0, 62, 23, 11, 51, 10, 50, 33, 4, 47, 28, 9, 22, 17, 69, 61, 26, 40, 6, 37, 22, 39, 5, 59, 17, 51, 2, 7, 26, 1, 59, 33, 60, 10, 58, 19, 54, 63, 51, 19, 31, 51, 50, 26, 12, 21, 34, 3, 19, 49, 2, 38, 33, 60, 9, 67, 55, 0, 17, 5, 64, 62, 53, 17, 9, 48, 39, 53, 32, 67, 6, 15, 28, 46, 48, 12, 29, 61, 4, 10, 66, 16, 21, 55, 2, 65, 13, 40, 43, 36, 46, 46, 72, 25, 50, 52, 66, 60, 66, 39, 29, 3, 49, 42, 32, 36, 70, 59, 55, 7, 68, 69, 67, 33, 19, 39, 4, 57, 63, 52, 47, 52, 37, 56, 73, 21, 21, 63, 3, 16, 59, 1, 31, 55, 56, 6, 27, 69, 14, 22, 55, 44, 70, 33, 52, 67, 56, 5, 13, 48, 24, 14, 49, 1, 62, 32, 48, 3, 51, 1, 38, 9, 71, 44, 29, 59, 38, 60, 12, 30, 52, 56, 73, 13, 15, 15, 37, 0, 61, 62, 32, 36, 62, 29, 65, 21, 71, 24, 57, 36, 72, 54, 57, 67, 61, 62, 36, 59, 6, 12, 33, 7, 31, 52, 4, 6, 47, 23, 66, 66, 16, 72, 1, 6, 7, 3, 32, 8, 51, 72, 0, 36, 66, 37, 56, 72, 9, 70, 4, 72, 37, 56, 11, 19, 26, 69, 20, 14, 53, 32, 25, 61, 0, 51, 34, 71, 20, 4, 69, 37, 57, 33, 39, 35, 57, 41, 28, 61, 12, 30, 63, 36, 73, 8, 52, 19, 20, 36, 50, 6, 17, 7, 59, 54, 58, 30, 2, 18, 8, 5, 72, 7, 54, 18, 7, 6, 21, 54, 66, 22, 12, 
22, 29, 27, 65, 25, 51, 67, 33, 69, 48, 6], 2020: [12, 20, 48, 61, 71, 62, 13, 63, 22, 65, 37, 15, 66, 71, 17, 3, 5, 44, 51, 10, 28, 36, 72, 65, 56, 6, 43, 30, 28, 13, 4, 51, 25, 49, 24, 43, 59, 10, 7, 67, 44, 16, 19, 1, 44, 9, 47, 71, 51, 58, 43, 35, 34, 44, 18, 21, 63, 5, 71, 47, 11, 73, 50, 67, 3, 59, 55, 62, 37, 60, 67, 68, 62, 51, 37, 26, 22, 66, 0, 57, 9, 29, 8, 38, 57, 30, 67, 52, 22, 49, 61, 44, 3, 53, 23, 1, 43, 71, 13, 49, 71, 63, 2, 51, 5, 25, 14, 3, 63, 50, 60, 60, 69, 10, 40, 3, 36, 17, 17, 23, 7, 14, 71, 52, 72, 0, 7, 45, 69, 65, 26, 66, 53, 21, 50, 13, 23, 66, 2, 1, 57, 8, 61, 7, 36, 69, 26, 2, 49, 35, 61, 7, 52, 13, 39, 49, 52, 14, 11, 31, 23, 17, 40, 14, 43, 57, 40, 48, 54, 62, 72, 67, 53, 7, 50, 43, 13, 36, 15, 43, 42, 62, 2, 43, 62, 23, 11, 59, 42, 47, 10, 2, 26, 36, 5, 21, 72, 67, 37, 50, 40, 55, 34, 36, 41, 64, 60, 21, 38, 41, 7, 23, 36, 6, 33, 50, 10, 57, 11, 53, 63, 51, 48, 31, 48, 46, 22, 48, 21, 73, 3, 16, 49, 65, 66, 35, 60, 8, 53, 49, 4, 63, 12, 55, 40, 53, 17, 72, 35, 71, 52, 70, 17, 3, 23, 28, 44, 11, 12, 65, 36, 1, 19, 39, 14, 11, 63, 5, 71, 17, 40, 42, 35, 37, 43, 32, 54, 47, 50, 35, 57, 21, 13, 27, 2, 47, 38, 50, 23, 70, 68, 32, 4, 68, 7, 67, 32, 70, 71, 57, 30, 69, 52, 43, 52, 35, 63, 33, 17, 21, 65, 3, 25, 56, 1, 37, 61, 56, 6, 24, 66, 14, 19, 52, 44, 70, 29, 19, 54, 34, 5, 1, 72, 21, 13, 72, 24, 52, 45, 48, 32, 60, 46, 66, 7, 68, 43, 24, 48, 59, 56, 54, 35, 52, 53, 70, 18, 56, 2, 12, 3, 58, 59, 32, 33, 30, 20, 38, 22, 71, 35, 48, 5, 72, 51, 19, 62, 58, 28, 40, 57, 25, 64, 33, 6, 28, 30, 11, 4, 47, 18, 53, 66, 14, 69, 54, 2, 31, 70, 32, 8, 50, 30, 40, 36, 19, 37, 65, 69, 9, 51, 1, 70, 33, 31, 11, 38, 8, 29, 20, 21, 1, 29, 48, 45, 25, 20, 31, 3, 20, 11, 69, 63, 43, 56, 56, 52, 55, 19, 29, 58, 8, 30, 21, 33, 23, 17, 34, 46, 71, 24, 48, 5, 64, 67, 59, 13, 3, 1, 20, 18, 6, 54, 69, 7, 55, 66, 43, 5, 24, 31, 0, 68, 11, 67, 29, 22, 65, 24, 0, 67, 10, 69, 69, 6], 2001: [12, 33, 50, 26, 27, 31, 41, 63, 6, 55, 40, 37, 44, 18, 23, 66, 25, 56, 54, 68, 28, 0, 3, 23, 56, 6, 44, 30, 71, 71, 56, 51, 15, 19, 26, 48, 50, 34, 55, 66, 45, 17, 19, 0, 30, 20, 47, 73, 53, 60, 43, 54, 41, 57, 25, 21, 41, 45, 64, 13, 9, 25, 59, 18, 73, 46, 66, 1, 37, 68, 19, 72, 65, 53, 7, 29, 65, 58, 73, 39, 56, 24, 9, 6, 61, 28, 68, 52, 13, 49, 5, 44, 6, 51, 63, 51, 51, 73, 40, 70, 72, 15, 54, 68, 64, 58, 19, 7, 62, 51, 52, 61, 69, 61, 54, 38, 54, 45, 17, 26, 71, 20, 26, 0, 73, 0, 10, 51, 30, 43, 64, 71, 54, 66, 73, 22, 10, 50, 25, 2, 5, 10, 61, 55, 73, 17, 28, 20, 41, 50, 64, 7, 55, 63, 39, 29, 0, 17, 11, 30, 22, 18, 61, 14, 14, 45, 57, 69, 23, 65, 70, 35, 41, 41, 6, 41, 20, 19, 15, 44, 35, 63, 35, 43, 62, 63, 39, 40, 8, 50, 33, 2, 59, 29, 9, 21, 63, 70, 35, 59, 40, 70, 38, 23, 39, 61, 62, 17, 53, 39, 23, 26, 37, 59, 37, 50, 13, 57, 20, 58, 30, 50, 53, 32, 50, 50, 22, 46, 43, 33, 3, 61, 18, 3, 36, 33, 72, 9, 67, 54, 12, 27, 10, 7, 68, 56, 35, 9, 36, 50, 53, 12, 60, 0, 14, 28, 47, 11, 12, 6, 36, 3, 21, 30, 46, 19, 55, 73, 65, 14, 60, 42, 23, 37, 46, 38, 26, 52, 54, 17, 73, 70, 39, 20, 4, 34, 40, 7, 37, 9, 39, 0, 6, 68, 68, 13, 30, 4, 7, 3, 57, 63, 53, 46, 11, 37, 55, 33, 21, 22, 70, 73, 20, 60, 1, 61, 55, 55, 6, 11, 58, 14, 22, 52, 44, 70, 33, 49, 56, 38, 6, 44, 26, 63, 44, 8, 5, 62, 46, 49, 3, 0, 49, 71, 1, 64, 57, 6, 60, 37, 58, 29, 28, 47, 57, 72, 19, 11, 15, 33, 0, 13, 38, 34, 33, 30, 39, 65, 33, 72, 24, 38, 6, 72, 54, 55, 18, 64, 12, 48, 58, 7, 6, 16, 9, 29, 51, 6, 9, 61, 23, 44, 66, 52, 73, 63, 10, 32, 31, 35, 63, 51, 40, 32, 41, 30, 42, 19, 72, 8, 51, 46, 3, 59, 32, 11, 55, 24, 44, 20, 28, 53, 33, 25, 39, 0, 21, 34, 4, 21, 4, 73, 32, 57, 26, 37, 35, 56, 41, 3, 60, 24, 71, 22, 48, 73, 19, 51, 46, 12, 36, 50, 11, 17, 8, 67, 50, 58, 30, 23, 70, 8, 4, 71, 6, 19, 71, 8, 6, 19, 32, 15, 61, 12, 0, 12, 22, 65, 25, 47, 1, 31, 56, 48, 20], 2039: [11, 72, 46, 61, 28, 62, 41, 61, 33, 37, 39, 17, 22, 18, 19, 50, 26, 47, 69, 72, 26, 39, 3, 67, 44, 6, 43, 30, 42, 13, 5, 51, 17, 19, 26, 65, 17, 48, 56, 67, 45, 17, 18, 1, 46, 10, 3, 73, 62, 66, 43, 24, 18, 20, 5, 51, 63, 44, 65, 14, 26, 1, 25, 27, 73, 46, 57, 57, 47, 68, 67, 34, 66, 53, 39, 28, 71, 24, 63, 59, 44, 1, 38, 6, 48, 30, 69, 52, 31, 19, 7, 46, 6, 67, 63, 50, 46, 52, 46, 52, 72, 15, 17, 71, 65, 1, 7, 55, 19, 1, 32, 50, 69, 28, 43, 37, 34, 64, 71, 69, 71, 16, 15, 0, 1, 42, 12, 47, 30, 21, 28, 58, 51, 3, 73, 23, 25, 67, 7, 34, 50, 13, 13, 18, 72, 16, 14, 73, 49, 35, 63, 7, 64, 15, 6, 65, 0, 70, 11, 11, 23, 20, 62, 14, 73, 66, 56, 69, 29, 67, 32, 36, 40, 40, 4, 3, 15, 17, 14, 16, 36, 63, 22, 1, 62, 23, 11, 51, 11, 49, 32, 1, 26, 30, 7, 23, 12, 69, 24, 26, 38, 29, 37, 22, 39, 62, 48, 0, 48, 43, 66, 26, 1, 58, 33, 66, 15, 59, 21, 56, 63, 53, 9, 31, 51, 0, 22, 12, 15, 35, 48, 18, 47, 2, 36, 32, 62, 7, 70, 51, 9, 65, 6, 61, 37, 53, 17, 48, 5, 8, 50, 71, 66, 3, 17, 29, 46, 10, 9, 31, 63, 4, 38, 66, 14, 52, 42, 8, 65, 13, 40, 0, 36, 40, 56, 71, 25, 51, 52, 37, 59, 66, 15, 67, 5, 49, 5, 43, 50, 70, 27, 55, 8, 69, 40, 67, 51, 19, 39, 2, 39, 69, 53, 45, 51, 37, 65, 48, 20, 15, 65, 3, 25, 59, 1, 31, 64, 58, 3, 27, 68, 15, 21, 54, 44, 59, 32, 21, 56, 56, 6, 8, 24, 23, 11, 49, 4, 61, 43, 50, 2, 0, 3, 40, 0, 37, 43, 25, 48, 59, 58, 52, 26, 52, 53, 73, 13, 13, 33, 34, 2, 61, 1, 32, 36, 30, 1, 64, 60, 71, 32, 37, 14, 2, 51, 24, 67, 61, 14, 19, 57, 27, 62, 33, 9, 28, 33, 4, 7, 47, 20, 21, 33, 16, 72, 1, 5, 31, 23, 34, 8, 50, 32, 33, 36, 30, 39, 56, 72, 9, 11, 19, 71, 56, 56, 7, 20, 10, 68, 20, 27, 53, 59, 26, 22, 73, 13, 26, 68, 27, 5, 16, 65, 57, 32, 36, 54, 60, 41, 31, 61, 11, 26, 23, 36, 72, 20, 54, 16, 20, 38, 49, 7, 58, 54, 27, 3, 6, 4, 4, 67, 30, 41, 71, 44, 17, 18, 62, 6, 38, 
41, 40, 22, 13, 61, 49, 29, 64, 26, 57, 68, 32, 57, 23, 26], 2006: [10, 33, 50, 60, 27, 31, 7, 64, 63, 28, 38, 12, 23, 17, 1, 67, 30, 21, 14, 71, 25, 35, 19, 29, 57, 70, 44, 38, 29, 73, 16, 29, 39, 24, 25, 35, 57, 48, 35, 66, 27, 17, 33, 0, 59, 35, 14, 72, 61, 62, 2, 23, 38, 21, 4, 49, 42, 4, 64, 52, 22, 0, 24, 17, 72, 29, 56, 57, 46, 67, 19, 73, 66, 52, 8, 27, 60, 58, 62, 39, 53, 20, 38, 51, 49, 52, 56, 41, 21, 20, 6, 49, 63, 68, 62, 2, 28, 53, 39, 72, 72, 14, 16, 70, 64, 26, 19, 65, 62, 0, 52, 61, 69, 11, 44, 40, 33, 28, 26, 68, 70, 7, 26, 9, 73, 41, 12, 56, 0, 66, 65, 68, 58, 20, 15, 22, 24, 72, 30, 42, 49, 12, 63, 55, 73, 17, 72, 19, 54, 34, 49, 7, 65, 15, 5, 26, 66, 65, 11, 12, 20, 23, 42, 16, 69, 45, 55, 68, 5, 27, 72, 32, 41, 39, 6, 44, 14, 19, 13, 42, 35, 3, 18, 0, 61, 24, 12, 39, 9, 4, 58, 2, 47, 29, 56, 19, 15, 68, 24, 7, 37, 60, 12, 37, 39, 61, 61, 18, 52, 42, 8, 42, 0, 59, 32, 52, 0, 59, 20, 1, 61, 49, 10, 32, 50, 66, 21, 47, 14, 20, 3, 17, 18, 5, 41, 34, 30, 10, 69, 56, 12, 15, 32, 10, 65, 3, 16, 8, 36, 55, 53, 29, 64, 4, 16, 29, 47, 49, 12, 30, 58, 2, 34, 15, 61, 19, 55, 72, 64, 12, 1, 43, 37, 39, 46, 41, 26, 51, 52, 36, 58, 64, 14, 17, 4, 44, 40, 70, 37, 70, 56, 63, 30, 68, 69, 14, 31, 32, 38, 1, 59, 3, 52, 44, 52, 37, 64, 73, 36, 19, 66, 62, 16, 60, 0, 29, 56, 56, 35, 28, 67, 15, 24, 41, 44, 27, 31, 49, 68, 55, 6, 13, 26, 62, 14, 67, 41, 60, 33, 49, 1, 61, 2, 39, 47, 48, 44, 30, 60, 23, 61, 25, 28, 52, 54, 72, 19, 65, 33, 38, 1, 60, 60, 36, 32, 31, 1, 65, 21, 72, 24, 57, 57, 11, 65, 55, 64, 59, 63, 37, 56, 26, 7, 32, 8, 32, 32, 4, 9, 48, 23, 65, 63, 16, 14, 63, 4, 7, 22, 34, 63, 54, 72, 34, 41, 29, 39, 66, 22, 8, 21, 18, 72, 11, 54, 8, 19, 24, 69, 49, 4, 32, 58, 25, 46, 72, 58, 29, 5, 21, 4, 67, 32, 41, 26, 36, 73, 67, 41, 25, 62, 13, 71, 22, 18, 17, 19, 50, 9, 63, 37, 49, 11, 17, 8, 54, 55, 7, 30, 7, 67, 30, 53, 11, 5, 69, 32, 63, 6, 18, 63, 2, 49, 12, 61, 50, 44, 63, 25, 54, 68, 31, 56, 27, 16], 2090: [13, 5, 70, 14, 27, 30, 43, 63, 51, 39, 47, 15, 22, 17, 24, 43, 26, 57, 10, 69, 28, 37, 2, 32, 31, 73, 24, 41, 44, 71, 5, 21, 19, 19, 6, 67, 19, 30, 38, 69, 50, 18, 18, 1, 46, 19, 48, 39, 16, 63, 0, 24, 38, 20, 20, 22, 46, 3, 65, 14, 12, 24, 50, 27, 7, 32, 4, 57, 46, 53, 18, 71, 66, 53, 39, 28, 65, 26, 34, 59, 46, 31, 12, 52, 48, 32, 57, 24, 13, 18, 63, 48, 10, 64, 4, 50, 21, 52, 48, 11, 71, 15, 40, 68, 67, 62, 37, 6, 21, 69, 20, 20, 70, 30, 54, 39, 36, 64, 25, 73, 22, 3, 27, 50, 73, 42, 12, 66, 31, 53, 64, 68, 20, 7, 60, 11, 11, 71, 61, 43, 24, 10, 65, 33, 64, 16, 29, 72, 10, 37, 50, 8, 55, 17, 6, 34, 69, 17, 12, 12, 23, 22, 62, 15, 19, 44, 49, 21, 31, 28, 6, 46, 55, 42, 4, 38, 17, 14, 16, 65, 36, 44, 36, 44, 64, 62, 61, 61, 4, 73, 68, 3, 56, 30, 9, 22, 25, 69, 38, 26, 40, 59, 37, 58, 22, 10, 50, 2, 73, 43, 64, 25, 37, 60, 71, 68, 3, 56, 66, 55, 30, 1, 52, 33, 51, 2, 58, 14, 6, 37, 4, 62, 49, 63, 39, 73, 16, 9, 72, 53, 11, 27, 33, 17, 35, 57, 34, 50, 5, 39, 54, 32, 63, 7, 17, 17, 47, 12, 13, 30, 37, 4, 32, 15, 62, 2, 56, 8, 65, 13, 4, 0, 23, 40, 45, 73, 56, 51, 54, 37, 9, 70, 39, 69, 18, 43, 41, 23, 36, 10, 30, 50, 8, 63, 21, 55, 34, 73, 39, 4, 26, 63, 24, 47, 50, 37, 28, 34, 35, 17, 67, 73, 17, 58, 10, 30, 30, 4, 1, 11, 68, 14, 70, 55, 45, 73, 32, 52, 65, 56, 5, 12, 48, 25, 66, 7, 44, 56, 45, 50, 64, 62, 3, 70, 64, 1, 57, 25, 63, 36, 60, 26, 49, 32, 57, 73, 15, 7, 70, 33, 0, 48, 38, 34, 38, 15, 34, 73, 32, 71, 27, 58, 45, 72, 0, 47, 18, 60, 64, 36, 59, 8, 5, 33, 11, 30, 35, 11, 7, 61, 27, 65, 66, 15, 17, 62, 7, 48, 3, 17, 63, 29, 4, 37, 36, 23, 46, 67, 72, 7, 22, 45, 54, 58, 33, 7, 39, 63, 67, 19, 62, 52, 34, 24, 4, 0, 22, 2, 72, 7, 5, 36, 33, 56, 27, 38, 35, 70, 42, 24, 61, 13, 28, 63, 68, 9, 19, 54, 47, 11, 25, 51, 14, 16, 9, 57, 0, 5, 6, 4, 38, 0, 4, 72, 5, 50, 16, 62, 7, 1, 33, 66, 68, 13, 8, 49, 31, 2, 26, 58, 67, 0, 57, 18, 28], 2057: [27, 72, 48, 61, 28, 61, 9, 63, 33, 65, 39, 15, 33, 18, 21, 50, 29, 44, 55, 69, 28, 44, 7, 28, 41, 6, 44, 30, 44, 72, 5, 51, 16, 19, 26, 49, 27, 10, 54, 67, 47, 16, 19, 1, 46, 8, 71, 73, 63, 63, 46, 62, 37, 44, 5, 12, 41, 46, 65, 50, 29, 26, 50, 70, 36, 28, 66, 66, 37, 69, 23, 32, 65, 53, 37, 28, 61, 26, 35, 38, 70, 1, 54, 6, 48, 29, 71, 52, 22, 19, 5, 48, 7, 53, 6, 1, 27, 16, 40, 52, 71, 15, 15, 69, 16, 25, 10, 7, 63, 39, 60, 60, 69, 22, 54, 3, 58, 17, 40, 72, 69, 20, 70, 20, 2, 42, 11, 3, 32, 65, 64, 15, 53, 66, 62, 57, 10, 66, 25, 41, 48, 10, 61, 44, 61, 73, 29, 25, 49, 49, 50, 7, 51, 59, 39, 65, 69, 64, 13, 0, 23, 62, 62, 67, 18, 39, 50, 70, 31, 66, 71, 35, 40, 38, 17, 31, 16, 15, 8, 13, 31, 62, 21, 0, 62, 23, 11, 38, 8, 51, 35, 2, 49, 28, 5, 25, 55, 69, 2, 6, 40, 11, 11, 36, 41, 62, 70, 1, 38, 43, 68, 25, 36, 57, 33, 51, 14, 60, 18, 58, 63, 34, 21, 31, 51, 46, 22, 48, 15, 73, 3, 27, 49, 4, 36, 35, 60, 51, 68, 51, 43, 63, 69, 64, 35, 52, 17, 41, 35, 54, 53, 9, 67, 6, 15, 28, 46, 63, 12, 29, 37, 4, 37, 66, 14, 3, 63, 2, 65, 13, 40, 42, 23, 40, 46, 36, 25, 53, 54, 35, 57, 21, 39, 65, 2, 5, 56, 50, 36, 69, 61, 36, 10, 68, 68, 71, 55, 18, 39, 4, 58, 62, 21, 47, 51, 35, 63, 62, 35, 17, 65, 3, 38, 59, 1, 30, 65, 55, 6, 11, 71, 14, 22, 40, 44, 69, 32, 53, 70, 56, 5, 36, 24, 63, 13, 66, 6, 63, 32, 48, 32, 62, 1, 37, 32, 68, 57, 6, 48, 38, 60, 53, 10, 51, 53, 73, 15, 17, 14, 37, 3, 63, 62, 35, 33, 30, 41, 61, 20, 71, 35, 57, 36, 35, 53, 13, 67, 63, 62, 18, 55, 25, 65, 15, 9, 31, 33, 6, 8, 47, 18, 53, 66, 47, 68, 5, 2, 7, 32, 32, 7, 55, 71, 33, 40, 66, 41, 56, 71, 9, 60, 45, 54, 12, 56, 11, 19, 26, 12, 20, 24, 52, 32, 48, 45, 73, 15, 3, 4, 20, 5, 69, 36, 40, 30, 35, 35, 14, 41, 28, 60, 12, 30, 21, 47, 36, 20, 33, 8, 18, 36, 47, 10, 16, 54, 59, 54, 58, 29, 58, 18, 
0, 54, 1, 7, 50, 18, 65, 2, 37, 50, 16, 67, 8, 22, 10, 26, 65, 4, 53, 67, 32, 69, 26, 15], 1985: [12, 70, 5, 61, 28, 64, 39, 58, 33, 65, 39, 68, 43, 9, 19, 50, 24, 46, 51, 12, 27, 44, 3, 31, 56, 22, 9, 30, 71, 15, 4, 51, 27, 49, 7, 49, 30, 58, 55, 70, 47, 17, 17, 16, 46, 36, 48, 73, 53, 66, 41, 62, 39, 53, 5, 12, 30, 42, 62, 49, 29, 1, 58, 45, 73, 31, 57, 66, 37, 68, 23, 35, 64, 60, 3, 33, 62, 56, 15, 38, 46, 31, 11, 50, 59, 51, 67, 52, 36, 15, 63, 60, 5, 53, 24, 2, 43, 73, 48, 52, 72, 64, 4, 68, 65, 59, 16, 3, 63, 37, 60, 63, 68, 22, 41, 12, 37, 43, 36, 25, 69, 16, 70, 0, 72, 0, 10, 45, 71, 38, 28, 67, 56, 24, 72, 23, 2, 39, 9, 22, 1, 13, 13, 53, 71, 17, 56, 60, 1, 0, 62, 6, 54, 64, 60, 65, 34, 32, 1, 31, 22, 31, 43, 14, 72, 57, 53, 69, 19, 27, 71, 36, 40, 41, 40, 31, 16, 15, 15, 42, 55, 62, 35, 56, 60, 23, 27, 59, 10, 72, 45, 4, 49, 25, 8, 21, 19, 70, 63, 26, 39, 29, 36, 36, 41, 63, 60, 0, 40, 2, 6, 26, 1, 59, 33, 24, 12, 57, 21, 54, 31, 50, 51, 31, 50, 1, 24, 48, 41, 73, 3, 61, 63, 2, 30, 35, 29, 51, 66, 54, 12, 19, 6, 7, 34, 52, 17, 41, 36, 38, 19, 11, 25, 6, 23, 27, 58, 48, 11, 64, 27, 49, 19, 66, 10, 3, 54, 7, 71, 49, 57, 42, 36, 27, 48, 34, 25, 46, 52, 35, 59, 21, 13, 27, 2, 49, 39, 42, 33, 9, 15, 57, 6, 66, 8, 67, 50, 18, 73, 50, 58, 62, 53, 45, 51, 37, 55, 73, 37, 20, 68, 3, 24, 59, 1, 61, 29, 43, 6, 27, 71, 17, 10, 53, 43, 69, 35, 21, 56, 36, 6, 48, 66, 37, 65, 66, 6, 61, 40, 63, 35, 0, 40, 37, 7, 70, 21, 8, 48, 28, 59, 53, 10, 32, 56, 72, 15, 48, 35, 12, 29, 61, 4, 34, 35, 30, 1, 72, 22, 70, 35, 57, 67, 71, 50, 57, 65, 60, 13, 39, 58, 26, 12, 33, 9, 31, 51, 8, 5, 47, 18, 55, 65, 16, 72, 1, 48, 57, 51, 34, 7, 49, 32, 33, 36, 66, 39, 58, 71, 26, 19, 47, 70, 35, 56, 10, 22, 9, 12, 20, 24, 53, 35, 25, 3, 44, 17, 33, 71, 23, 10, 71, 65, 1, 58, 38, 53, 14, 41, 22, 60, 33, 23, 23, 35, 35, 20, 54, 48, 20, 38, 50, 6, 66, 54, 27, 8, 6, 29, 23, 71, 7, 4, 71, 46, 55, 67, 6, 14, 35, 32, 38, 70, 11, 67, 10, 22, 64, 24, 14, 68, 32, 66, 21, 16], 2049: [25, 72, 49, 25, 24, 10, 41, 27, 49, 73, 39, 9, 46, 18, 19, 5, 25, 22, 53, 72, 73, 48, 3, 22, 58, 72, 45, 40, 54, 71, 20, 53, 17, 51, 25, 57, 61, 18, 58, 68, 47, 17, 19, 1, 46, 12, 3, 73, 55, 61, 1, 38, 41, 21, 26, 48, 42, 47, 63, 51, 1, 1, 58, 27, 73, 26, 57, 55, 39, 69, 18, 35, 66, 52, 31, 26, 65, 24, 35, 38, 55, 1, 38, 6, 61, 30, 56, 30, 13, 19, 5, 46, 6, 70, 24, 50, 46, 73, 43, 52, 0, 31, 4, 54, 65, 1, 18, 55, 66, 52, 68, 61, 69, 11, 43, 37, 40, 44, 71, 0, 9, 18, 25, 0, 72, 42, 12, 47, 32, 48, 62, 70, 57, 66, 15, 21, 10, 67, 23, 34, 53, 13, 34, 54, 72, 17, 27, 19, 43, 47, 63, 7, 55, 15, 39, 26, 67, 16, 10, 11, 23, 47, 62, 16, 73, 66, 59, 22, 7, 67, 72, 70, 40, 50, 4, 3, 23, 18, 17, 45, 53, 63, 35, 1, 62, 23, 68, 60, 11, 49, 46, 4, 61, 28, 28, 23, 6, 72, 24, 26, 65, 12, 37, 22, 39, 31, 61, 0, 48, 41, 7, 26, 1, 53, 33, 54, 4, 59, 19, 56, 63, 51, 19, 31, 51, 48, 33, 12, 21, 35, 48, 19, 20, 59, 40, 32, 72, 9, 66, 55, 9, 17, 21, 9, 70, 53, 37, 48, 48, 52, 53, 11, 64, 6, 13, 30, 46, 48, 12, 29, 17, 4, 60, 57, 16, 21, 57, 2, 65, 13, 34, 58, 36, 40, 46, 71, 25, 50, 52, 19, 73, 66, 13, 67, 3, 52, 42, 34, 36, 9, 71, 55, 8, 64, 3, 14, 16, 71, 39, 5, 33, 69, 53, 48, 14, 37, 66, 73, 20, 15, 65, 3, 16, 59, 9, 31, 55, 58, 6, 11, 73, 15, 21, 54, 44, 72, 66, 51, 67, 57, 60, 8, 16, 23, 35, 46, 1, 61, 46, 57, 35, 0, 48, 70, 47, 20, 44, 51, 60, 59, 60, 25, 38, 49, 58, 73, 13, 15, 14, 34, 0, 61, 62, 32, 36, 32, 41, 64, 18, 71, 23, 57, 65, 14, 53, 57, 67, 61, 14, 73, 53, 73, 73, 15, 11, 28, 51, 56, 6, 60, 1, 68, 54, 16, 72, 65, 10, 31, 32, 8, 29, 53, 30, 62, 38, 26, 39, 66, 1, 30, 11, 43, 71, 65, 32, 8, 42, 23, 68, 20, 24, 53, 35, 25, 35, 0, 22, 36, 4, 20, 12, 66, 65, 57, 58, 36, 54, 60, 41, 51, 62, 11, 73, 63, 36, 30, 18, 41, 27, 73, 36, 50, 8, 17, 7, 29, 15, 6, 4, 2, 67, 7, 41, 72, 4, 56, 32, 62, 6, 15, 41, 2, 22, 11, 69, 31, 24, 64, 25, 51, 68, 30, 57, 23, 19], 2206: [15, 72, 49, 25, 59, 33, 43, 27, 8, 68, 39, 51, 7, 73, 4, 64, 27, 20, 53, 13, 32, 19, 21, 0, 71, 69, 0, 40, 53, 3, 18, 26, 19, 17, 17, 30, 52, 37, 58, 65, 19, 19, 32, 35, 46, 12, 67, 44, 55, 67, 37, 66, 40, 68, 21, 2, 32, 64, 63, 10, 14, 26, 58, 48, 19, 71, 59, 5, 45, 69, 18, 70, 65, 51, 31, 41, 59, 56, 55, 48, 9, 35, 11, 38, 48, 28, 56, 56, 26, 67, 33, 73, 6, 10, 24, 50, 20, 27, 56, 52, 0, 19, 26, 19, 63, 55, 8, 6, 14, 73, 71, 45, 69, 38, 43, 3, 40, 66, 55, 3, 21, 18, 25, 11, 63, 40, 50, 47, 51, 68, 62, 52, 57, 53, 15, 20, 13, 69, 29, 42, 23, 17, 32, 54, 72, 58, 47, 63, 43, 47, 63, 7, 67, 17, 43, 32, 67, 16, 71, 11, 39, 45, 41, 16, 68, 33, 54, 73, 21, 69, 4, 70, 40, 7, 50, 1, 23, 18, 45, 73, 42, 5, 5, 19, 62, 63, 39, 59, 65, 3, 67, 1, 14, 28, 9, 12, 23, 67, 26, 71, 73, 0, 37, 46, 41, 52, 60, 29, 51, 41, 7, 10, 35, 51, 73, 54, 16, 22, 19, 0, 34, 51, 52, 50, 51, 12, 26, 50, 21, 37, 16, 19, 12, 7, 40, 2, 16, 25, 71, 55, 50, 39, 41, 9, 62, 34, 17, 7, 38, 73, 56, 32, 19, 16, 13, 10, 44, 28, 12, 4, 17, 4, 31, 14, 60, 0, 27, 71, 63, 39, 49, 45, 36, 39, 64, 5, 57, 50, 22, 21, 71, 0, 37, 67, 3, 43, 42, 5, 36, 73, 71, 62, 22, 71, 3, 11, 45, 7, 7, 30, 47, 26, 24, 70, 54, 64, 66, 51, 65, 21, 63, 6, 27, 57, 27, 61, 53, 3, 34, 9, 66, 4, 3, 30, 44, 26, 23, 52, 67, 45, 60, 8, 66, 61, 25, 6, 9, 14, 31, 64, 37, 60, 4, 69, 0, 71, 43, 29, 59, 59, 60, 24, 14, 15, 0, 73, 59, 60, 51, 35, 0, 51, 36, 60, 36, 33, 3, 42, 18, 71, 18, 19, 71, 8, 51, 19, 50, 61, 30, 36, 50, 73, 4, 13, 9, 31, 34, 54, 6, 4, 26, 47, 60, 51, 72, 58, 9, 60, 7, 53, 11, 53, 38, 40, 51, 3, 39, 11, 56, 45, 11, 45, 72, 6, 72, 68, 54, 11, 15, 34, 3, 31, 32, 48, 58, 0, 22, 2, 4, 6, 11, 66, 31, 60, 25, 38, 33, 54, 1, 51, 43, 12, 16, 21, 36, 39, 62, 49, 49, 22, 24, 52, 14, 67, 7, 
27, 64, 46, 4, 52, 27, 72, 43, 72, 4, 18, 34, 44, 5, 60, 54, 0, 22, 14, 69, 8, 24, 65, 25, 45, 3, 13, 69, 24, 19], 2056: [67, 71, 47, 26, 24, 31, 40, 25, 6, 55, 44, 33, 52, 23, 6, 67, 25, 30, 19, 71, 5, 44, 7, 23, 54, 28, 43, 42, 71, 41, 66, 5, 32, 24, 7, 9, 8, 15, 57, 66, 47, 17, 17, 0, 32, 42, 57, 43, 53, 67, 29, 20, 40, 57, 48, 31, 14, 29, 53, 28, 9, 25, 64, 58, 6, 47, 57, 52, 37, 64, 19, 70, 65, 49, 5, 33, 70, 58, 31, 39, 50, 35, 28, 2, 60, 29, 72, 21, 9, 17, 33, 65, 7, 52, 52, 51, 56, 73, 54, 72, 72, 29, 4, 5, 66, 30, 16, 7, 62, 41, 69, 17, 5, 45, 43, 17, 49, 42, 52, 0, 9, 52, 26, 56, 2, 38, 11, 61, 28, 39, 69, 68, 52, 60, 15, 18, 4, 50, 24, 2, 9, 15, 13, 55, 51, 22, 25, 69, 35, 46, 8, 7, 55, 63, 53, 48, 24, 13, 71, 9, 20, 63, 66, 16, 56, 31, 57, 30, 5, 66, 48, 70, 46, 39, 6, 12, 21, 16, 13, 45, 40, 0, 36, 39, 26, 63, 48, 0, 10, 28, 67, 17, 53, 5, 9, 29, 71, 66, 63, 62, 56, 66, 36, 62, 40, 66, 49, 18, 42, 40, 16, 42, 37, 19, 24, 51, 14, 58, 32, 28, 32, 17, 51, 20, 50, 50, 24, 47, 42, 34, 3, 20, 18, 40, 38, 31, 72, 6, 52, 53, 8, 39, 17, 60, 62, 65, 31, 47, 6, 50, 49, 29, 25, 26, 21, 28, 69, 9, 12, 5, 17, 3, 21, 57, 43, 24, 55, 35, 69, 19, 58, 47, 11, 9, 24, 70, 7, 50, 52, 58, 73, 11, 44, 65, 27, 1, 40, 19, 35, 70, 27, 60, 37, 44, 69, 72, 31, 1, 7, 23, 29, 68, 17, 46, 11, 37, 15, 24, 16, 21, 64, 62, 25, 72, 5, 61, 18, 1, 2, 51, 59, 16, 72, 41, 44, 70, 31, 23, 65, 43, 8, 31, 26, 37, 40, 8, 10, 61, 68, 63, 35, 22, 4, 69, 44, 65, 35, 39, 59, 28, 59, 68, 25, 44, 59, 72, 29, 15, 67, 33, 1, 45, 52, 57, 37, 4, 29, 70, 33, 2, 24, 57, 73, 23, 7, 9, 63, 60, 71, 1, 62, 12, 12, 18, 8, 7, 52, 10, 65, 62, 26, 20, 66, 52, 36, 56, 9, 1, 5, 52, 68, 51, 1, 56, 41, 68, 68, 30, 24, 27, 21, 6, 11, 37, 20, 66, 41, 64, 57, 12, 20, 64, 31, 70, 39, 72, 55, 53, 3, 4, 9, 73, 66, 73, 59, 55, 54, 57, 35, 11, 62, 24, 25, 18, 67, 62, 19, 47, 60, 12, 40, 50, 13, 73, 8, 63, 55, 58, 30, 52, 23, 70, 41, 71, 11, 54, 72, 7, 6, 37, 41, 2, 59, 9, 66, 6, 66, 65, 16, 33, 1, 73, 25, 22, 25], 2042: [12, 72, 69, 61, 72, 63, 7, 59, 32, 65, 40, 16, 32, 53, 23, 51, 14, 56, 13, 13, 70, 38, 19, 29, 44, 7, 43, 30, 71, 14, 63, 52, 17, 20, 26, 50, 60, 48, 55, 68, 45, 16, 19, 0, 44, 34, 47, 38, 61, 64, 43, 54, 34, 21, 21, 52, 64, 47, 14, 17, 27, 23, 51, 14, 73, 44, 67, 60, 37, 69, 67, 34, 62, 0, 3, 29, 63, 54, 35, 39, 9, 30, 9, 65, 31, 30, 68, 52, 26, 19, 64, 41, 46, 65, 3, 2, 20, 12, 47, 50, 72, 63, 2, 70, 68, 23, 8, 65, 20, 65, 60, 61, 69, 23, 40, 38, 37, 17, 26, 69, 70, 15, 71, 21, 72, 22, 12, 5, 30, 21, 28, 58, 51, 66, 15, 13, 10, 67, 8, 40, 31, 11, 13, 16, 72, 17, 72, 73, 9, 47, 61, 7, 65, 65, 5, 64, 35, 67, 11, 32, 24, 18, 41, 14, 66, 45, 51, 2, 54, 27, 72, 37, 60, 43, 41, 37, 38, 17, 15, 64, 32, 63, 22, 2, 62, 23, 11, 40, 9, 53, 10, 2, 27, 26, 6, 21, 4, 70, 37, 26, 37, 30, 28, 37, 39, 63, 60, 17, 48, 41, 7, 26, 0, 59, 33, 57, 13, 57, 19, 54, 63, 51, 10, 32, 50, 48, 22, 12, 21, 35, 3, 10, 49, 5, 36, 28, 57, 9, 69, 52, 12, 45, 32, 6, 32, 53, 62, 72, 36, 36, 53, 9, 64, 4, 16, 28, 47, 11, 12, 65, 61, 3, 38, 64, 61, 53, 43, 3, 4, 14, 40, 43, 36, 17, 46, 72, 23, 47, 52, 36, 60, 23, 40, 67, 0, 2, 39, 72, 34, 70, 56, 53, 4, 68, 7, 68, 51, 19, 72, 5, 59, 3, 52, 48, 52, 37, 64, 73, 23, 18, 63, 3, 16, 57, 2, 31, 55, 53, 6, 28, 72, 15, 24, 52, 44, 70, 33, 22, 64, 57, 6, 13, 64, 24, 14, 47, 7, 60, 46, 49, 3, 51, 1, 38, 6, 71, 44, 8, 48, 59, 59, 55, 27, 53, 54, 70, 14, 12, 2, 11, 3, 59, 73, 33, 33, 30, 41, 65, 22, 68, 33, 57, 6, 69, 51, 55, 67, 61, 14, 20, 56, 7, 28, 34, 7, 29, 31, 4, 6, 48, 16, 66, 66, 16, 70, 63, 6, 32, 23, 72, 8, 51, 30, 34, 72, 67, 37, 56, 69, 8, 11, 2, 55, 56, 56, 8, 38, 24, 69, 18, 54, 53, 35, 25, 35, 45, 16, 32, 69, 21, 4, 7, 34, 57, 33, 36, 54, 52, 41, 29, 60, 9, 27, 62, 27, 37, 18, 55, 46, 71, 24, 49, 6, 64, 55, 56, 4, 6, 30, 8, 72, 30, 3, 11, 44, 18, 15, 6, 12, 21, 30, 64, 69, 12, 42, 50, 21, 65, 22, 55, 68, 33, 67, 27, 6], 2094: [11, 71, 50, 27, 27, 53, 12, 61, 22, 36, 45, 15, 22, 35, 22, 50, 29, 55, 69, 69, 28, 36, 16, 24, 55, 5, 42, 30, 44, 73, 64, 51, 27, 19, 26, 45, 38, 39, 38, 69, 44, 8, 18, 1, 73, 33, 18, 73, 53, 63, 3, 53, 38, 22, 51, 50, 41, 4, 65, 14, 34, 0, 25, 27, 73, 59, 66, 57, 47, 65, 20, 37, 65, 53, 33, 28, 61, 71, 2, 59, 51, 69, 11, 52, 48, 30, 26, 52, 15, 20, 38, 50, 8, 66, 14, 52, 28, 70, 48, 73, 71, 15, 0, 68, 67, 57, 13, 8, 21, 67, 20, 63, 68, 28, 54, 13, 36, 16, 25, 44, 71, 20, 69, 20, 73, 42, 7, 15, 6, 55, 64, 14, 53, 23, 58, 22, 22, 71, 25, 40, 50, 10, 61, 33, 73, 18, 73, 72, 10, 37, 50, 7, 66, 64, 39, 32, 0, 67, 12, 31, 23, 22, 62, 13, 28, 42, 49, 69, 23, 67, 32, 35, 42, 40, 6, 3, 15, 20, 14, 43, 36, 54, 21, 42, 64, 25, 39, 40, 6, 51, 41, 3, 58, 30, 9, 20, 12, 69, 36, 8, 38, 73, 39, 36, 41, 64, 50, 19, 52, 43, 6, 25, 37, 55, 69, 66, 10, 56, 66, 24, 62, 50, 11, 33, 51, 14, 22, 47, 17, 65, 18, 21, 47, 2, 36, 34, 32, 11, 72, 51, 11, 16, 23, 10, 35, 72, 17, 28, 35, 41, 52, 8, 65, 5, 17, 30, 57, 10, 11, 19, 59, 4, 33, 39, 14, 2, 56, 73, 65, 13, 40, 0, 35, 40, 58, 71, 53, 51, 54, 65, 59, 23, 39, 66, 5, 39, 5, 7, 49, 8, 57, 0, 31, 68, 68, 3, 31, 2, 39, 4, 73, 69, 26, 47, 48, 37, 55, 34, 21, 17, 71, 63, 25, 58, 1, 30, 26, 43, 36, 11, 68, 17, 70, 55, 44, 30, 32, 21, 69, 58, 5, 10, 24, 63, 66, 72, 1, 58, 47, 50, 2, 62, 46, 37, 5, 73, 45, 29, 60, 61, 58, 26, 26, 52, 55, 73, 17, 28, 33, 36, 2, 13, 38, 34, 32, 29, 1, 66, 34, 71, 34, 37, 62, 72, 56, 56, 64, 60, 66, 18, 57, 41, 64, 33, 9, 28, 56, 11, 7, 47, 27, 21, 66, 70, 64, 62, 7, 46, 3, 49, 64, 55, 52, 42, 36, 69, 41, 67, 7, 9, 24, 19, 56, 56, 55, 7, 39, 25, 67, 19, 9, 65, 34, 26, 47, 73, 13, 31, 67, 27, 5, 16, 33, 68, 27, 38, 35, 55, 38, 31, 60, 13, 28, 23, 63, 72, 20, 51, 45, 20,
14, 49, 9, 69, 54, 55, 0, 5, 4, 61, 18, 52, 41, 68, 5, 20, 16, 8, 6, 19, 41, 65, 68, 10, 61, 1, 22, 64, 61, 57, 67, 32, 69, 20, 26], 1942: [10, 6, 47, 60, 28, 43, 6, 62, 32, 64, 40, 14, 23, 18, 23, 25, 30, 44, 54, 13, 27, 36, 19, 29, 56, 19, 43, 30, 71, 16, 4, 50, 27, 19, 26, 45, 59, 48, 55, 66, 43, 15, 19, 0, 46, 13, 14, 73, 63, 64, 3, 71, 37, 21, 5, 12, 42, 4, 65, 15, 24, 1, 26, 26, 64, 29, 65, 60, 37, 68, 19, 35, 61, 53, 2, 29, 62, 25, 35, 38, 67, 30, 55, 6, 49, 29, 54, 52, 21, 19, 5, 44, 63, 68, 63, 2, 23, 53, 48, 70, 70, 14, 1, 68, 68, 25, 19, 7, 33, 65, 52, 49, 69, 21, 19, 38, 35, 28, 17, 26, 18, 37, 70, 19, 72, 0, 10, 3, 45, 50, 25, 58, 60, 20, 73, 12, 10, 72, 5, 35, 28, 10, 3, 34, 72, 17, 72, 73, 48, 38, 51, 7, 67, 64, 5, 26, 18, 67, 12, 31, 22, 23, 61, 14, 65, 45, 23, 69, 5, 27, 43, 36, 41, 41, 41, 37, 16, 19, 15, 56, 35, 62, 35, 1, 62, 22, 10, 31, 7, 50, 10, 2, 45, 29, 4, 21, 12, 70, 2, 26, 39, 70, 10, 36, 39, 61, 59, 14, 38, 43, 67, 26, 0, 59, 33, 55, 0, 57, 41, 54, 63, 50, 8, 31, 50, 1, 22, 47, 20, 22, 3, 15, 48, 3, 36, 34, 60, 9, 69, 52, 12, 15, 2, 64, 34, 3, 2, 8, 36, 36, 53, 29, 66, 4, 16, 30, 47, 11, 12, 31, 36, 3, 68, 62, 61, 3, 55, 3, 49, 14, 16, 42, 35, 45, 46, 72, 26, 53, 52, 34, 60, 21, 39, 30, 5, 14, 40, 7, 37, 70, 56, 55, 32, 68, 38, 67, 54, 4, 38, 3, 27, 0, 52, 50, 51, 36, 55, 73, 20, 18, 66, 3, 16, 60, 0, 31, 64, 59, 6, 27, 69, 13, 22, 54, 44, 60, 33, 53, 64, 55, 4, 13, 45, 24, 65, 66, 6, 62, 46, 47, 53, 63, 31, 37, 6, 57, 44, 6, 47, 23, 26, 13, 28, 51, 54, 72, 14, 71, 33, 11, 3, 13, 72, 34, 33, 30, 41, 61, 21, 72, 35, 35, 5, 71, 54, 55, 67, 61, 61, 18, 58, 24, 12, 32, 3, 31, 31, 6, 8, 46, 18, 66, 63, 16, 73, 63, 6, 7, 2, 73, 8, 49, 72, 32, 35, 18, 42, 56, 20, 8, 51, 1, 70, 17, 56, 10, 20, 24, 68, 20, 15, 53, 33, 25, 46, 73, 51, 3, 68, 20, 4, 69, 34, 57, 28, 26, 73, 14, 41, 28, 60, 12, 29, 62, 45, 35, 20, 52, 16, 62, 36, 48, 10, 17, 8, 67, 1, 6, 29, 9, 18, 5, 4, 13, 7, 38, 17, 6, 6, 18, 32, 65, 68, 11, 61, 57, 22, 65, 25, 14, 68, 32, 56, 19, 16], 1999: [12, 72, 49, 64, 25, 10, 41, 30, 10, 55, 39, 37, 44, 33, 4, 44, 27, 56, 32, 13, 29, 44, 3, 33, 0, 9, 43, 30, 71, 71, 56, 17, 15, 22, 7, 37, 62, 37, 57, 70, 47, 21, 19, 0, 46, 34, 69, 73, 15, 61, 1, 38, 39, 21, 21, 48, 18, 64, 64, 51, 32, 1, 23, 30, 73, 46, 69, 55, 37, 68, 18, 72, 66, 53, 7, 29, 65, 0, 35, 38, 56, 1, 11, 6, 61, 28, 70, 25, 13, 19, 5, 44, 6, 65, 24, 50, 46, 73, 68, 52, 0, 18, 54, 47, 65, 62, 18, 7, 66, 54, 47, 61, 69, 1, 43, 3, 37, 14, 55, 26, 21, 18, 28, 0, 1, 0, 10, 47, 32, 68, 64, 71, 56, 66, 73, 7, 10, 50, 25, 2, 42, 13, 64, 54, 72, 17, 29, 19, 52, 38, 7, 7, 55, 67, 19, 48, 28, 18, 71, 11, 36, 20, 65, 14, 43, 56, 54, 22, 21, 65, 14, 47, 2, 43, 53, 39, 20, 17, 17, 43, 44, 47, 35, 45, 62, 63, 31, 62, 11, 50, 68, 4, 59, 27, 8, 23, 6, 70, 39, 58, 73, 70, 37, 46, 41, 32, 48, 17, 51, 2, 60, 26, 36, 51, 36, 55, 13, 57, 19, 56, 35, 53, 51, 50, 50, 48, 57, 50, 43, 55, 19, 19, 20, 3, 40, 32, 72, 11, 53, 54, 7, 39, 24, 9, 36, 56, 17, 26, 38, 50, 53, 32, 64, 6, 15, 7, 46, 62, 12, 4, 40, 3, 21, 30, 63, 55, 57, 72, 65, 14, 34, 45, 22, 40, 56, 35, 16, 50, 52, 17, 73, 23, 39, 30, 3, 34, 42, 1, 50, 12, 71, 35, 8, 64, 3, 51, 45, 1, 37, 5, 58, 15, 21, 48, 51, 23, 66, 22, 20, 15, 31, 6, 16, 59, 7, 31, 55, 0, 6, 11, 58, 17, 73, 54, 44, 60, 33, 53, 56, 36, 63, 8, 27, 26, 44, 6, 42, 62, 46, 64, 35, 0, 49, 40, 57, 71, 2, 14, 60, 61, 60, 27, 46, 50, 58, 72, 59, 17, 18, 34, 0, 61, 38, 58, 36, 30, 23, 64, 32, 7, 35, 38, 9, 5, 54, 22, 67, 61, 17, 36, 60, 39, 6, 36, 9, 31, 51, 41, 6, 61, 27, 66, 69, 16, 73, 65, 8, 49, 32, 35, 12, 55, 50, 60, 39, 22, 39, 19, 10, 30, 21, 4, 72, 59, 56, 8, 42, 64, 68, 20, 32, 53, 35, 25, 64, 0, 19, 34, 3, 30, 4, 72, 65, 57, 58, 41, 72, 71, 41, 62, 62, 11, 16, 66, 36, 66, 20, 57, 46, 12, 36, 50, 10, 17, 7, 27, 72, 6, 4, 2, 27, 7, 7, 72, 4, 58, 34, 62, 6, 18, 44, 0, 22, 14, 0, 52, 24, 65, 25, 51, 1, 33, 57, 23, 6], 2125: [9, 14, 46, 61, 28, 30, 7, 63, 7, 38, 37, 15, 51, 8, 22, 43, 26, 45, 16, 69, 29, 37, 19, 29, 43, 73, 43, 31, 68, 17, 57, 17, 16, 20, 26, 63, 28, 34, 55, 66, 44, 18, 20, 1, 46, 21, 3, 7, 60, 65, 0, 24, 38, 54, 6, 51, 42, 4, 67, 51, 26, 24, 50, 27, 30, 59, 66, 20, 49, 69, 18, 73, 66, 55, 39, 28, 70, 1, 73, 38, 57, 29, 9, 7, 31, 29, 68, 53, 9, 19, 5, 44, 63, 64, 63, 2, 23, 72, 18, 70, 72, 16, 16, 70, 67, 58, 19, 64, 21, 66, 20, 62, 69, 20, 55, 39, 38, 28, 18, 69, 18, 21, 62, 9, 72, 21, 12, 70, 8, 21, 65, 16, 55, 66, 15, 73, 10, 71, 26, 3, 58, 10, 61, 33, 72, 17, 29, 20, 53, 9, 50, 4, 56, 25, 40, 51, 69, 66, 14, 29, 37, 18, 62, 67, 65, 44, 51, 68, 5, 67, 72, 36, 60, 62, 5, 3, 17, 18, 16, 41, 36, 63, 17, 1, 64, 23, 40, 61, 8, 52, 37, 3, 72, 26, 6, 21, 71, 69, 3, 26, 41, 12, 7, 37, 39, 62, 70, 1, 39, 40, 20, 26, 0, 59, 33, 51, 14, 58, 65, 55, 63, 45, 50, 32, 49, 20, 23, 47, 44, 34, 3, 62, 49, 4, 37, 30, 60, 9, 68, 53, 11, 27, 6, 65, 35, 4, 18, 47, 36, 54, 53, 29, 68, 4, 17, 28, 47, 12, 12, 31, 37, 68, 62, 31, 61, 11, 64, 73, 65, 13, 41, 42, 48, 37, 46, 37, 55, 50, 13, 35, 18, 71, 40, 65, 3, 35, 39, 23, 34, 70, 26, 59, 20, 68, 69, 8, 34, 5, 39, 4, 38, 68, 52, 47, 52, 36, 65, 73, 22, 19, 60, 3, 17, 72, 1, 31, 55, 56, 21, 12, 71, 15, 23, 53, 41, 70, 33, 49, 71, 56, 5, 6, 25, 64, 14, 66, 6, 60, 33, 48, 52, 50, 31, 38, 11, 65, 58, 7, 25, 38, 57, 12, 50, 52, 58, 71, 15, 57, 16, 37, 1, 14, 72, 32, 34, 31, 1, 62, 21, 6, 36, 39, 18, 73, 55, 17, 67, 65, 13, 49, 59, 43, 66, 16, 7, 29, 31, 4, 9, 62, 19, 54, 63, 15, 70, 62, 7, 31, 22, 51, 8, 51, 72, 31, 41, 30, 42, 30, 72, 7, 52, 9, 54, 68, 33, 12, 39, 24, 69, 20, 53, 53, 35, 49, 61, 0, 20, 25, 4, 21, 5, 69, 33, 57, 27, 36, 53, 19, 41, 25, 61, 13, 30, 22, 49, 36, 21, 34, 16,
20, 37, 48, 11, 17, 23, 68, 54, 58, 30, 0, 66, 29, 4, 10, 7, 16, 18, 62, 6, 20, 41, 2, 69, 12, 0, 11, 27, 65, 26, 55, 68, 16, 70, 27, 16], 2074: [13, 72, 50, 61, 24, 31, 7, 60, 33, 65, 43, 28, 33, 17, 15, 48, 33, 20, 53, 14, 28, 34, 4, 30, 43, 20, 44, 28, 42, 72, 17, 5, 40, 23, 24, 34, 9, 48, 58, 67, 0, 16, 18, 1, 46, 10, 48, 44, 61, 61, 43, 54, 18, 22, 49, 52, 43, 44, 14, 16, 23, 24, 27, 27, 30, 47, 57, 56, 46, 69, 18, 34, 66, 53, 3, 28, 63, 26, 35, 59, 68, 29, 64, 38, 48, 30, 55, 40, 22, 19, 37, 45, 7, 62, 25, 50, 27, 52, 48, 71, 71, 15, 17, 70, 69, 59, 8, 53, 32, 41, 20, 62, 69, 22, 20, 39, 38, 64, 53, 69, 19, 38, 26, 52, 73, 0, 12, 47, 30, 21, 36, 68, 57, 21, 13, 13, 27, 73, 24, 24, 31, 10, 59, 35, 73, 16, 12, 20, 53, 47, 50, 7, 55, 15, 6, 65, 69, 66, 11, 12, 23, 24, 62, 67, 66, 46, 51, 1, 6, 67, 72, 33, 40, 63, 50, 40, 15, 19, 16, 51, 36, 63, 36, 0, 62, 23, 11, 59, 6, 51, 8, 3, 70, 26, 9, 22, 21, 69, 24, 6, 38, 59, 11, 58, 40, 6, 60, 18, 49, 43, 16, 42, 1, 51, 33, 51, 14, 58, 65, 55, 62, 51, 9, 50, 51, 2, 58, 48, 21, 35, 48, 20, 49, 4, 34, 33, 60, 9, 68, 55, 4, 28, 6, 9, 37, 54, 15, 48, 6, 39, 73, 32, 65, 42, 17, 28, 44, 49, 13, 31, 65, 4, 35, 72, 12, 2, 55, 4, 65, 13, 2, 0, 65, 40, 57, 73, 65, 51, 53, 35, 61, 67, 13, 67, 4, 50, 40, 3, 34, 8, 57, 59, 7, 65, 20, 15, 51, 5, 39, 4, 30, 4, 52, 47, 52, 37, 56, 73, 17, 14, 65, 3, 17, 72, 1, 30, 55, 1, 32, 11, 68, 16, 23, 55, 44, 28, 31, 52, 67, 56, 3, 14, 46, 32, 10, 66, 40, 63, 47, 50, 66, 62, 1, 70, 7, 60, 43, 27, 59, 59, 60, 26, 28, 52, 53, 73, 15, 15, 16, 35, 1, 45, 60, 3, 37, 63, 41, 21, 22, 71, 24, 32, 18, 72, 55, 17, 67, 61, 14, 36, 59, 27, 13, 33, 7, 29, 52, 4, 6, 58, 24, 65, 66, 16, 64, 62, 7, 7, 23, 51, 63, 55, 36, 33, 71, 68, 43, 54, 72, 9, 52, 19, 54, 56, 31, 7, 19, 24, 69, 20, 12, 1, 32, 26, 35, 71, 12, 26, 4, 25, 5, 16, 33, 58, 34, 37, 54, 55, 42, 26, 61, 11, 30, 63, 27, 72, 19, 53, 17, 20, 36, 49, 7, 17, 7, 57, 54, 7, 13, 60, 66, 29, 41, 72, 5, 18, 18, 62, 6, 39, 41, 3, 23, 12, 61, 10, 23, 66, 26, 53, 68, 31, 67, 24, 19], 2031: [12, 33, 46, 26, 28, 33, 39, 62, 33, 65, 41, 64, 22, 29, 21, 66, 29, 44, 15, 14, 4, 40, 3, 28, 43, 4, 0, 30, 71, 72, 15, 51, 38, 19, 7, 49, 58, 10, 39, 70, 47, 14, 17, 0, 72, 20, 18, 42, 53, 40, 3, 54, 39, 57, 23, 50, 41, 5, 60, 56, 29, 25, 49, 18, 73, 24, 66, 66, 47, 68, 23, 72, 40, 53, 7, 33, 61, 58, 63, 38, 50, 19, 11, 6, 59, 28, 38, 52, 26, 67, 50, 50, 62, 67, 5, 51, 22, 73, 40, 69, 72, 15, 54, 4, 68, 45, 18, 7, 43, 39, 56, 63, 71, 61, 54, 38, 54, 15, 51, 27, 69, 20, 50, 0, 2, 0, 11, 3, 34, 38, 64, 72, 56, 0, 17, 22, 6, 36, 25, 44, 29, 19, 60, 34, 61, 17, 73, 25, 39, 49, 51, 3, 69, 63, 39, 65, 0, 18, 16, 0, 41, 2, 66, 14, 51, 50, 50, 69, 23, 66, 35, 72, 58, 61, 17, 41, 20, 15, 54, 44, 55, 54, 35, 15, 62, 63, 58, 57, 7, 51, 45, 4, 57, 1, 8, 20, 16, 30, 35, 58, 44, 34, 38, 53, 41, 37, 70, 20, 53, 39, 12, 43, 37, 18, 37, 55, 14, 57, 61, 58, 15, 57, 21, 31, 50, 50, 22, 46, 19, 73, 3, 9, 45, 58, 36, 35, 72, 27, 0, 54, 12, 63, 30, 11, 36, 58, 39, 9, 35, 50, 57, 9, 60, 6, 21, 12, 48, 59, 1, 6, 62, 3, 33, 16, 14, 3, 29, 73, 69, 23, 40, 43, 69, 15, 48, 36, 2, 52, 24, 23, 73, 23, 39, 65, 5, 13, 61, 7, 35, 70, 61, 0, 31, 64, 68, 13, 30, 3, 7, 3, 58, 4, 25, 46, 11, 37, 56, 62, 21, 20, 70, 3, 71, 61, 1, 61, 55, 55, 36, 11, 71, 17, 41, 28, 40, 69, 32, 21, 56, 38, 6, 44, 13, 63, 44, 3, 48, 61, 42, 54, 2, 65, 4, 71, 1, 7, 57, 29, 60, 57, 59, 29, 10, 65, 8, 72, 19, 17, 53, 12, 1, 13, 38, 33, 33, 30, 39, 65, 20, 72, 35, 61, 35, 10, 38, 57, 67, 60, 65, 48, 62, 25, 6, 33, 9, 31, 52, 6, 9, 47, 22, 53, 62, 52, 68, 63, 8, 45, 32, 49, 4, 51, 40, 33, 36, 34, 41, 42, 24, 8, 21, 19, 56, 37, 22, 10, 55, 64, 12, 20, 28, 49, 34, 25, 60, 73, 19, 30, 63, 20, 9, 68, 70, 72, 63, 30, 35, 56, 41, 3, 62, 24, 58, 23, 49, 73, 20, 51, 46, 12, 36, 50, 10, 0, 22, 67, 12, 8, 73, 52, 71, 53, 54, 71, 6, 53, 17, 27, 6, 19, 56, 34, 55, 10, 1, 10, 26, 64, 4, 47, 1, 41, 25, 47, 20], 2088: [11, 69, 6, 27, 28, 62, 10, 62, 11, 65, 36, 15, 50, 30, 17, 50, 56, 44, 59, 69, 28, 36, 24, 24, 55, 5, 43, 30, 44, 16, 29, 51, 37, 19, 26, 45, 59, 10, 54, 66, 45, 8, 18, 1, 30, 37, 47, 70, 51, 63, 43, 63, 36, 22, 24, 51, 15, 4, 71, 50, 25, 26, 27, 44, 29, 59, 66, 1, 34, 65, 67, 73, 61, 50, 2, 32, 60, 30, 2, 35, 51, 69, 11, 6, 48, 28, 67, 52, 22, 0, 5, 44, 3, 71, 6, 1, 23, 52, 40, 70, 71, 15, 1, 68, 69, 24, 14, 6, 63, 65, 32, 36, 69, 15, 54, 13, 36, 16, 52, 26, 6, 0, 71, 20, 73, 55, 58, 73, 48, 65, 64, 68, 53, 56, 49, 12, 22, 37, 25, 0, 29, 10, 61, 35, 64, 69, 70, 2, 49, 50, 50, 7, 55, 64, 39, 29, 18, 67, 14, 31, 23, 22, 62, 43, 71, 56, 49, 69, 31, 62, 10, 35, 59, 61, 41, 42, 21, 14, 4, 56, 36, 1, 21, 42, 62, 23, 63, 40, 8, 12, 10, 48, 26, 28, 5, 20, 20, 69, 36, 2, 40, 6, 33, 23, 41, 64, 50, 14, 38, 40, 6, 25, 36, 48, 33, 50, 10, 60, 41, 58, 30, 50, 8, 47, 51, 45, 22, 47, 17, 52, 3, 26, 49, 23, 36, 34, 60, 23, 50, 51, 4, 27, 5, 64, 68, 30, 17, 54, 35, 39, 52, 8, 67, 3, 17, 28, 47, 59, 12, 64, 61, 0, 10, 30, 14, 22, 63, 8, 65, 22, 40, 42, 35, 37, 56, 37, 26, 53, 54, 34, 60, 20, 39, 20, 2, 14, 38, 2, 49, 70, 39, 1, 32, 68, 6, 67, 59, 4, 39, 4, 57, 63, 52, 47, 52, 35, 63, 19, 21, 17, 71, 3, 25, 60, 1, 30, 61, 56, 5, 11, 70, 14, 18, 51, 44, 70, 32, 21, 70, 56, 5, 32, 24, 63, 13, 47, 39, 62, 47, 61, 6, 64, 46, 37, 6, 51, 40, 11, 48, 23, 26, 62, 45, 47, 57, 73, 17, 16, 18, 12, 0, 58, 58, 58, 33, 22, 38, 61, 34, 71, 35, 38, 62, 72, 54, 56, 53, 64, 66, 49, 57, 25, 64, 16, 6, 31, 34, 11, 9, 47, 23, 53, 66, 70, 69, 62, 1, 7, 3, 32, 8, 49, 2, 32, 36, 19, 42, 56, 72, 9, 71, 19, 70, 16, 55, 11, 19, 8, 29, 19, 14, 52, 34, 26, 61, 24, 18, 34, 68, 20, 5, 69, 35, 68, 29, 31, 52, 55, 38, 28, 39, 12, 30, 63,
63, 63, 20, 52, 61, 17, 36, 47, 5, 16, 13, 59, 13, 5, 30, 8, 18, 0, 54, 68, 7, 54, 18, 66, 1, 13, 68, 27, 68, 10, 22, 29, 27, 65, 61, 52, 67, 9, 69, 20, 15], 2037: [27, 72, 48, 56, 25, 61, 67, 37, 33, 13, 43, 11, 33, 44, 15, 0, 12, 42, 69, 72, 24, 34, 14, 27, 43, 20, 65, 28, 66, 72, 43, 64, 27, 53, 24, 49, 36, 38, 34, 67, 43, 8, 12, 1, 46, 8, 45, 12, 57, 44, 2, 22, 14, 59, 3, 52, 26, 36, 69, 67, 60, 1, 47, 30, 30, 28, 19, 47, 16, 15, 23, 13, 17, 53, 37, 26, 4, 62, 35, 54, 49, 1, 59, 50, 53, 40, 71, 60, 34, 19, 37, 36, 1, 21, 61, 57, 27, 52, 48, 47, 42, 35, 49, 0, 65, 18, 17, 53, 32, 37, 41, 60, 60, 22, 20, 53, 21, 67, 56, 30, 13, 38, 67, 39, 44, 9, 5, 3, 38, 50, 20, 53, 59, 64, 10, 20, 27, 0, 31, 1, 25, 8, 59, 42, 11, 17, 12, 48, 45, 35, 48, 7, 18, 51, 57, 65, 69, 66, 9, 18, 19, 24, 9, 44, 10, 46, 61, 58, 27, 18, 15, 33, 62, 38, 17, 55, 34, 25, 12, 65, 34, 2, 19, 15, 68, 23, 52, 38, 31, 10, 8, 12, 70, 1, 65, 18, 11, 67, 1, 0, 34, 29, 22, 58, 4, 60, 49, 43, 24, 43, 72, 23, 10, 62, 28, 47, 27, 59, 65, 32, 41, 34, 25, 54, 51, 46, 20, 42, 15, 18, 47, 27, 45, 4, 57, 52, 43, 51, 68, 56, 43, 12, 69, 62, 72, 7, 2, 27, 6, 54, 23, 16, 66, 1, 50, 30, 44, 19, 16, 31, 57, 4, 62, 66, 60, 23, 61, 4, 65, 11, 38, 64, 73, 40, 57, 14, 42, 66, 67, 64, 57, 21, 13, 49, 67, 50, 3, 44, 65, 9, 40, 12, 12, 66, 4, 62, 20, 5, 66, 47, 58, 0, 66, 61, 20, 57, 55, 62, 51, 67, 65, 1, 38, 26, 71, 51, 59, 1, 63, 15, 68, 5, 7, 40, 1, 28, 45, 57, 59, 31, 71, 67, 7, 41, 35, 66, 40, 11, 31, 59, 10, 30, 31, 12, 32, 55, 34, 5, 44, 35, 0, 70, 6, 69, 58, 73, 15, 17, 62, 53, 7, 11, 0, 3, 31, 28, 19, 21, 32, 71, 10, 20, 63, 35, 55, 13, 67, 71, 64, 24, 55, 5, 63, 24, 4, 26, 55, 65, 7, 42, 3, 19, 25, 16, 63, 11, 70, 3, 32, 73, 42, 18, 51, 33, 32, 28, 43, 42, 18, 31, 58, 51, 54, 67, 11, 9, 66, 68, 27, 38, 12, 20, 44, 8, 43, 40, 54, 3, 4, 0, 54, 42, 38, 43, 34, 38, 72, 66, 19, 26, 60, 11, 53, 18, 45, 32, 3, 3, 8, 62, 34, 45, 36, 51, 50, 65, 54, 8, 4, 9, 71, 31, 8, 7, 13, 68, 16, 63, 29, 39, 66, 3, 67, 8, 42, 10, 48, 10, 4, 53, 65, 72, 6, 72, 13], 2084: [56, 33, 69, 26, 24, 62, 7, 61, 33, 2, 40, 12, 34, 19, 21, 25, 33, 43, 53, 73, 25, 35, 4, 23, 44, 4, 46, 28, 44, 73, 55, 20, 14, 52, 26, 57, 54, 44, 66, 69, 41, 8, 18, 0, 47, 12, 13, 38, 63, 45, 3, 23, 19, 22, 51, 52, 42, 4, 64, 18, 33, 2, 27, 31, 63, 47, 19, 70, 35, 69, 19, 36, 65, 53, 32, 27, 65, 60, 1, 39, 50, 29, 64, 51, 54, 30, 72, 40, 26, 30, 33, 22, 7, 73, 3, 51, 47, 53, 44, 52, 43, 19, 13, 70, 66, 23, 8, 7, 62, 9, 69, 58, 69, 17, 43, 13, 33, 27, 72, 23, 9, 24, 68, 17, 73, 42, 12, 47, 30, 55, 63, 58, 19, 4, 15, 21, 21, 70, 24, 24, 26, 14, 2, 54, 73, 17, 28, 44, 43, 47, 64, 7, 68, 63, 6, 65, 69, 67, 13, 35, 20, 26, 10, 67, 32, 67, 54, 1, 28, 18, 72, 35, 41, 39, 37, 68, 16, 19, 13, 51, 35, 63, 20, 0, 63, 23, 57, 31, 6, 51, 33, 2, 44, 29, 47, 24, 12, 68, 61, 7, 39, 30, 11, 5, 40, 58, 49, 18, 36, 43, 62, 42, 1, 59, 19, 65, 0, 60, 66, 1, 64, 61, 26, 32, 51, 30, 26, 13, 42, 35, 48, 46, 49, 20, 27, 33, 60, 10, 69, 56, 44, 17, 70, 28, 70, 8, 2, 27, 6, 3, 73, 72, 65, 0, 16, 30, 46, 49, 9, 61, 58, 4, 68, 67, 61, 24, 55, 73, 66, 13, 2, 30, 23, 41, 46, 36, 25, 50, 53, 65, 73, 67, 14, 50, 3, 52, 42, 7, 37, 67, 56, 55, 8, 65, 4, 67, 51, 62, 7, 3, 39, 4, 52, 49, 49, 37, 56, 49, 21, 16, 66, 62, 25, 60, 3, 70, 65, 1, 35, 28, 70, 11, 52, 55, 44, 70, 31, 52, 68, 58, 61, 9, 48, 67, 11, 71, 10, 32, 47, 58, 52, 1, 36, 41, 1, 56, 35, 29, 59, 36, 61, 54, 25, 70, 59, 73, 19, 15, 33, 35, 1, 45, 62, 33, 37, 62, 41, 65, 33, 72, 24, 32, 51, 14, 55, 56, 68, 62, 65, 48, 51, 6, 64, 34, 4, 27, 52, 0, 6, 52, 4, 20, 66, 17, 64, 51, 6, 7, 33, 36, 63, 55, 52, 34, 72, 63, 40, 56, 69, 46, 71, 18, 55, 16, 57, 9, 19, 24, 69, 21, 72, 21, 46, 26, 39, 72, 12, 26, 67, 26, 4, 73, 37, 44, 59, 39, 54, 12, 39, 51, 61, 12, 30, 19, 37, 73, 19, 52, 17, 63, 36, 51, 9, 17, 7, 57, 0, 7, 5, 10, 72, 0, 41, 72, 11, 69, 18, 63, 6, 18, 41, 29, 23, 9, 61, 14, 26, 65, 65, 33, 13, 33, 56, 24, 32], 2137: [12, 23, 47, 13, 29, 32, 38, 66, 7, 65, 40, 37, 53, 30, 20, 65, 28, 47, 18, 13, 31, 43, 3, 27, 59, 23, 10, 33, 53, 16, 32, 54, 37, 22, 37, 48, 62, 13, 38, 64, 27, 18, 31, 2, 30, 37, 17, 41, 53, 66, 46, 38, 39, 56, 46, 73, 40, 41, 44, 49, 32, 25, 32, 67, 73, 46, 69, 65, 37, 68, 43, 71, 64, 16, 6, 32, 65, 25, 54, 62, 56, 34, 12, 38, 59, 28, 70, 55, 25, 5, 32, 44, 6, 71, 9, 50, 26, 73, 39, 53, 72, 18, 5, 60, 5, 28, 17, 65, 52, 53, 50, 53, 21, 21, 44, 52, 53, 26, 2, 26, 38, 17, 49, 23, 1, 0, 10, 73, 72, 49, 63, 18, 52, 59, 73, 24, 3, 49, 28, 23, 32, 13, 64, 34, 46, 72, 57, 2, 52, 38, 26, 7, 55, 16, 42, 17, 21, 22, 2, 17, 45, 21, 65, 17, 71, 56, 60, 34, 26, 65, 24, 71, 2, 4, 7, 63, 19, 14, 7, 43, 30, 53, 35, 46, 65, 43, 28, 52, 11, 73, 32, 51, 48, 27, 8, 24, 15, 73, 62, 5, 43, 58, 37, 57, 40, 64, 48, 17, 41, 38, 60, 29, 36, 59, 72, 25, 13, 57, 60, 57, 14, 54, 52, 45, 54, 2, 25, 5, 18, 15, 36, 19, 21, 3, 39, 32, 63, 72, 67, 54, 1, 46, 5, 10, 43, 64, 7, 45, 37, 49, 56, 12, 43, 7, 55, 18, 50, 62, 12, 65, 61, 50, 32, 18, 17, 22, 55, 8, 68, 17, 43, 45, 22, 60, 56, 50, 26, 56, 53, 38, 60, 20, 38, 30, 56, 47, 40, 73, 26, 70, 60, 58, 35, 69, 9, 70, 36, 0, 6, 3, 57, 63, 54, 46, 10, 38, 65, 25, 20, 20, 31, 66, 28, 60, 4, 61, 30, 17, 1, 37, 58, 3, 73, 44, 44, 73, 69, 24, 73, 37, 9, 67, 27, 62, 9, 2, 5, 62, 67, 29, 35, 0, 3, 38, 45, 71, 56, 28, 59, 39, 59, 52, 52, 47, 57, 72, 14, 26, 18, 57, 0, 50, 37, 58, 36, 60, 2, 64, 47, 27, 38, 58, 8, 34, 15, 8, 66, 60, 64, 21, 61, 27, 5, 16, 8, 3, 51, 9, 19, 50, 21, 56, 66, 51, 16, 4, 48, 49, 4, 53, 11, 50, 35, 32, 39, 22, 40, 36, 72, 70, 22, 4, 71, 36, 31, 27, 23, 32, 11, 11, 9, 48, 31, 71, 64, 1, 18, 34, 3, 30, 8, 72, 66, 71, 59, 34, 52, 55, 22, 23,
 61, 23, 57, 62, 36, 26, 9, 39, 7, 21, 39, 50, 6, 20, 36, 62, 53, 58, 3, 2, 0, 7, 5, 71, 47, 52, 68, 6, 1, 11, 33, 0, 47, 7, 70, 32, 23, 65, 26, 57, 69, 12, 17, 21, 17], 1967: [11, 33, 73, 60, 73, 64, 12, 61, 6, 55, 39, 17, 22, 29, 19, 24, 28, 46, 15, 12, 71, 39, 3, 67, 44, 18, 44, 32, 43, 15, 4, 53, 27, 51, 72, 48, 58, 48, 55, 67, 73, 17, 16, 1, 46, 36, 48, 73, 61, 66, 42, 53, 18, 25, 5, 50, 40, 8, 73, 49, 28, 25, 25, 18, 73, 32, 57, 65, 47, 68, 19, 47, 63, 53, 1, 28, 61, 25, 59, 59, 46, 31, 10, 50, 32, 27, 69, 52, 25, 4, 7, 43, 5, 66, 59, 46, 51, 52, 48, 72, 72, 64, 0, 68, 64, 37, 17, 66, 19, 51, 20, 50, 68, 21, 44, 2, 39, 26, 50, 25, 38, 16, 49, 22, 73, 22, 9, 72, 29, 49, 24, 68, 51, 55, 72, 23, 25, 39, 9, 34, 29, 13, 59, 9, 73, 71, 73, 21, 51, 37, 62, 6, 54, 15, 60, 25, 0, 70, 12, 31, 23, 1, 43, 65, 70, 50, 52, 69, 6, 27, 32, 33, 39, 63, 15, 41, 15, 19, 6, 55, 29, 7, 18, 3, 61, 31, 61, 51, 10, 49, 32, 4, 28, 27, 7, 23, 60, 69, 35, 4, 38, 4, 2, 56, 40, 64, 47, 16, 8, 43, 66, 43, 1, 59, 40, 52, 12, 59, 21, 60, 62, 32, 11, 30, 50, 29, 24, 12, 14, 60, 3, 18, 13, 2, 30, 56, 27, 9, 66, 51, 6, 65, 65, 10, 34, 32, 16, 37, 48, 8, 34, 71, 65, 5, 54, 29, 44, 47, 11, 35, 60, 3, 35, 65, 16, 52, 54, 8, 64, 13, 15, 42, 22, 44, 4, 34, 2, 52, 52, 65, 59, 65, 17, 29, 1, 13, 40, 7, 35, 9, 60, 54, 35, 42, 8, 69, 58, 56, 73, 2, 39, 4, 53, 45, 34, 37, 55, 73, 21, 20, 70, 73, 15, 59, 1, 60, 7, 54, 36, 26, 68, 16, 20, 54, 43, 59, 35, 21, 72, 61, 8, 67, 46, 23, 6, 66, 38, 61, 66, 50, 34, 0, 31, 37, 7, 7, 69, 28, 58, 36, 58, 29, 28, 32, 54, 72, 19, 13, 65, 12, 2, 61, 60, 34, 35, 60, 39, 61, 60, 70, 34, 57, 34, 71, 56, 56, 34, 60, 61, 21, 51, 44, 56, 33, 8, 2, 32, 6, 5, 49, 21, 65, 65, 16, 72, 3, 4, 7, 3, 34, 63, 51, 22, 32, 38, 69, 41, 45, 24, 9, 70, 43, 5, 35, 55, 9, 20, 47, 11, 19, 12, 52, 59, 26, 22, 7, 50, 33, 67, 23, 5, 68, 38, 1, 32, 38, 54, 55, 40, 23, 60, 10, 58, 23, 27, 72, 19, 54, 17, 20, 38, 49, 7, 60, 54, 61, 20, 8, 3, 61, 67, 7, 4, 12, 6, 36, 67, 5, 36, 35, 32, 65, 22, 7, 60, 49, 31, 64, 25, 57, 68, 31, 68, 28, 16], 2116: [11, 71, 71, 12, 28, 31, 40, 65, 6, 55, 39, 17, 52, 9, 19, 69, 25, 1, 19, 71, 30, 17, 21, 31, 44, 22, 66, 40, 52, 68, 18, 51, 73, 18, 29, 42, 61, 13, 57, 63, 58, 17, 30, 1, 32, 18, 41, 72, 53, 57, 42, 26, 35, 55, 21, 48, 30, 1, 43, 65, 7, 26, 71, 63, 72, 69, 57, 21, 36, 19, 66, 70, 63, 49, 5, 27, 46, 58, 53, 58, 48, 33, 11, 5, 58, 28, 38, 54, 24, 2, 2, 60, 5, 65, 57, 49, 45, 73, 40, 72, 10, 17, 68, 24, 4, 28, 13, 66, 52, 67, 46, 52, 71, 43, 43, 13, 36, 25, 55, 68, 21, 2, 53, 22, 73, 36, 57, 72, 34, 39, 62, 17, 50, 58, 72, 16, 19, 39, 27, 34, 49, 12, 63, 15, 45, 71, 5, 19, 51, 10, 62, 6, 69, 15, 38, 16, 63, 67, 16, 33, 23, 20, 64, 16, 42, 53, 53, 16, 10, 64, 26, 26, 39, 3, 6, 39, 18, 16, 43, 42, 38, 44, 36, 45, 61, 63, 27, 60, 10, 49, 31, 50, 14, 32, 7, 73, 65, 33, 21, 28, 42, 34, 36, 56, 40, 63, 47, 16, 40, 39, 59, 25, 35, 50, 22, 9, 2, 58, 61, 28, 62, 30, 51, 44, 53, 66, 24, 4, 51, 69, 3, 18, 51, 23, 38, 31, 62, 71, 52, 51, 6, 45, 4, 9, 36, 6, 6, 37, 38, 65, 55, 31, 69, 6, 19, 27, 11, 62, 26, 64, 70, 2, 31, 24, 16, 52, 65, 62, 72, 19, 34, 45, 47, 39, 32, 40, 63, 70, 52, 21, 27, 63, 14, 45, 71, 46, 40, 31, 52, 70, 36, 60, 34, 68, 8, 50, 61, 56, 5, 21, 73, 2, 53, 44, 54, 37, 64, 21, 19, 14, 73, 16, 16, 60, 1, 60, 29, 16, 71, 54, 73, 2, 72, 59, 43, 59, 28, 21, 63, 45, 8, 66, 26, 23, 6, 49, 38, 61, 66, 63, 35, 64, 2, 39, 44, 48, 55, 25, 50, 66, 59, 15, 51, 44, 54, 45, 59, 11, 17, 12, 6, 61, 4, 57, 35, 65, 1, 70, 31, 5, 26, 24, 59, 43, 56, 49, 34, 60, 63, 39, 60, 26, 11, 18, 8, 31, 51, 8, 38, 49, 26, 55, 65, 72, 36, 56, 47, 10, 3, 34, 8, 49, 26, 32, 36, 21, 39, 58, 22, 44, 21, 43, 72, 20, 55, 26, 37, 64, 28, 66, 8, 47, 31, 12, 63, 7, 53, 43, 2, 23, 7, 71, 35, 65, 29, 28, 54, 12, 43, 31, 62, 24, 58, 23, 65, 71, 19, 54, 48, 20, 26, 49, 13, 19, 6, 27, 15, 57, 3, 52, 72, 7, 41, 70, 9, 56, 20, 25, 6, 12, 6, 32, 59, 14, 60, 31, 24, 63, 25, 33, 67, 11, 16, 22, 18], 2114: [15, 71, 49, 18, 24, 10, 40, 65, 33, 68, 34, 64, 52, 73, 26, 69, 25, 1, 53, 71, 30, 38, 21, 31, 44, 22, 46, 40, 44, 68, 18, 19, 54, 18, 7, 57, 53, 37, 57, 63, 47, 19, 30, 12, 32, 18, 41, 72, 63, 67, 45, 26, 40, 22, 21, 2, 30, 1, 5, 49, 14, 26, 22, 58, 6, 47, 57, 21, 37, 52, 19, 70, 13, 52, 5, 27, 65, 25, 62, 39, 48, 33, 11, 0, 60, 52, 56, 54, 24, 2, 2, 41, 7, 73, 4, 49, 20, 53, 44, 72, 10, 17, 68, 70, 66, 56, 22, 7, 52, 67, 63, 39, 40, 50, 43, 13, 14, 39, 55, 3, 21, 18, 25, 11, 73, 41, 11, 54, 5, 50, 63, 68, 19, 34, 15, 21, 13, 69, 24, 24, 49, 12, 64, 54, 45, 20, 28, 19, 12, 48, 64, 7, 67, 17, 39, 16, 22, 67, 14, 35, 20, 20, 64, 16, 68, 53, 57, 71, 31, 66, 53, 70, 41, 39, 4, 2, 23, 16, 36, 42, 35, 5, 36, 46, 61, 63, 70, 60, 6, 51, 67, 50, 42, 32, 9, 24, 65, 33, 6, 28, 42, 1, 36, 58, 40, 63, 49, 18, 51, 42, 10, 42, 35, 61, 22, 9, 2, 60, 19, 28, 32, 30, 51, 32, 51, 66, 24, 71, 42, 56, 58, 46, 49, 23, 38, 31, 55, 25, 52, 55, 8, 45, 47, 9, 44, 56, 16, 27, 38, 49, 36, 12, 58, 24, 19, 30, 6, 9, 12, 64, 17, 4, 31, 19, 18, 24, 55, 10, 64, 19, 34, 45, 22, 39, 65, 36, 10, 50, 23, 64, 27, 14, 14, 45, 17, 1, 42, 6, 49, 70, 71, 60, 29, 71, 3, 50, 33, 1, 41, 59, 39, 2, 52, 46, 54, 37, 66, 21, 19, 16, 73, 6, 25, 60, 3, 61, 29, 1, 34, 10, 70, 4, 72, 55, 44, 70, 70, 52, 67, 45, 8, 53, 66, 37, 26, 6, 36, 10, 68, 52, 3, 61, 2, 69, 0, 48, 56, 27, 27, 36, 60, 25, 25, 44, 59, 73, 59, 27, 37, 35, 33, 45, 37, 60, 37, 65, 32, 42, 18, 5, 26, 59, 47, 72, 68, 56, 66, 61, 63, 4, 56, 6, 22, 43, 8, 31, 34, 10, 6, 38, 26, 55, 66, 72, 36, 61, 9, 25, 5, 43, 11, 54, 51, 33, 40, 21, 39, 24, 22, 44, 21, 43, 72, 20, 32, 69, 41, 11, 69, 12, 20, 64, 31, 26, 37, 72, 12, 43, 2, 19, 12, 66, 66, 70, 
 59, 39, 54, 55, 1, 51, 43, 12, 42, 62, 67, 37, 19, 50, 49, 22, 40, 50, 13, 20, 7, 57, 54, 7, 3, 32, 72, 51, 41, 72, 4, 56, 32, 7, 6, 17, 54, 32, 59, 12, 69, 31, 24, 65, 25, 57, 4, 13, 56, 24, 18], 2027: [13, 14, 72, 27, 28, 53, 40, 42, 12, 65, 30, 15, 22, 35, 2, 65, 48, 2, 19, 69, 3, 41, 22, 24, 68, 23, 10, 43, 53, 22, 19, 51, 37, 19, 8, 50, 49, 35, 38, 67, 28, 16, 18, 1, 35, 19, 18, 41, 16, 65, 46, 63, 38, 30, 24, 13, 31, 50, 44, 50, 30, 26, 55, 27, 73, 59, 66, 6, 38, 53, 20, 71, 39, 48, 6, 30, 63, 66, 73, 35, 51, 69, 12, 6, 60, 29, 26, 52, 22, 23, 5, 61, 8, 53, 14, 50, 23, 54, 40, 70, 71, 15, 55, 15, 69, 57, 19, 8, 63, 71, 57, 54, 72, 34, 54, 13, 36, 42, 52, 9, 38, 20, 71, 1, 18, 38, 12, 50, 35, 51, 64, 68, 53, 23, 50, 17, 5, 37, 25, 45, 29, 10, 61, 39, 69, 23, 24, 2, 49, 67, 52, 7, 55, 64, 39, 43, 23, 19, 72, 32, 23, 3, 62, 26, 28, 57, 57, 1, 31, 62, 1, 73, 42, 61, 54, 40, 19, 16, 8, 43, 36, 63, 36, 46, 68, 23, 40, 63, 36, 73, 68, 3, 15, 26, 9, 22, 45, 71, 36, 26, 40, 59, 37, 37, 41, 64, 50, 14, 13, 40, 68, 25, 36, 43, 23, 52, 3, 58, 60, 24, 28, 50, 52, 31, 51, 51, 25, 47, 42, 70, 36, 24, 19, 65, 54, 39, 60, 28, 68, 53, 13, 63, 50, 11, 37, 59, 40, 9, 37, 51, 56, 8, 59, 25, 17, 28, 2, 1, 27, 65, 28, 4, 10, 63, 44, 4, 56, 8, 69, 20, 61, 0, 48, 40, 69, 23, 17, 58, 55, 22, 73, 64, 39, 66, 2, 47, 13, 2, 38, 70, 16, 52, 7, 68, 17, 12, 36, 55, 39, 4, 30, 63, 52, 47, 52, 37, 56, 38, 17, 14, 69, 38, 25, 61, 1, 30, 8, 17, 3, 11, 71, 14, 18, 46, 44, 70, 32, 21, 65, 37, 5, 49, 27, 63, 66, 2, 11, 62, 10, 54, 3, 62, 5, 70, 16, 49, 24, 29, 59, 62, 60, 54, 54, 52, 55, 73, 60, 28, 6, 33, 61, 58, 18, 32, 23, 4, 38, 38, 34, 73, 27, 25, 60, 72, 2, 72, 64, 60, 31, 40, 63, 25, 13, 33, 6, 32, 52, 11, 10, 47, 27, 66, 66, 70, 69, 62, 7, 58, 22, 53, 12, 50, 72, 52, 36, 34, 37, 59, 23, 9, 29, 5, 4, 71, 21, 11, 40, 65, 58, 57, 21, 65, 34, 71, 4, 25, 51, 39, 35, 21, 5, 44, 33, 72, 27, 36, 35, 55, 44, 11, 61, 25, 30, 63, 36, 72, 65, 37, 49, 20, 27, 49, 11, 20, 11, 59, 51, 58, 30, 28, 10, 52, 41, 68, 7, 52, 18, 7, 6, 13, 41, 0, 21, 31, 67, 29, 65, 65, 1, 0, 67, 32, 69, 20, 19], 2123: [11, 71, 72, 61, 71, 62, 40, 42, 53, 65, 40, 15, 66, 10, 26, 50, 33, 2, 18, 13, 3, 18, 22, 53, 68, 23, 45, 57, 53, 16, 19, 51, 37, 7, 8, 49, 49, 59, 38, 73, 48, 16, 18, 1, 73, 19, 17, 41, 53, 67, 46, 26, 36, 56, 24, 13, 31, 65, 44, 50, 46, 26, 22, 1, 70, 23, 57, 51, 38, 69, 67, 57, 39, 3, 3, 32, 63, 0, 34, 59, 8, 31, 11, 68, 58, 29, 71, 52, 22, 18, 2, 61, 6, 66, 14, 50, 52, 52, 40, 55, 71, 15, 40, 15, 69, 22, 17, 6, 52, 67, 63, 52, 72, 22, 43, 13, 36, 26, 40, 26, 24, 53, 71, 20, 73, 14, 11, 30, 35, 38, 63, 68, 53, 23, 73, 17, 13, 40, 63, 45, 4, 13, 64, 35, 64, 20, 73, 2, 2, 35, 62, 7, 55, 68, 39, 30, 23, 22, 17, 17, 23, 28, 64, 67, 71, 57, 53, 1, 31, 62, 49, 27, 40, 7, 41, 40, 19, 14, 53, 43, 31, 63, 36, 46, 68, 23, 61, 59, 11, 51, 32, 5, 42, 26, 7, 22, 45, 71, 36, 18, 40, 59, 37, 37, 41, 64, 59, 47, 13, 8, 60, 25, 49, 62, 23, 51, 3, 58, 15, 57, 14, 48, 52, 31, 51, 18, 25, 45, 42, 23, 36, 10, 66, 65, 38, 65, 71, 6, 72, 51, 58, 46, 5, 9, 37, 59, 40, 27, 39, 39, 56, 21, 61, 5, 23, 28, 2, 62, 12, 65, 28, 4, 10, 15, 46, 4, 55, 8, 71, 20, 35, 43, 35, 37, 49, 41, 6, 53, 52, 64, 60, 24, 29, 45, 2, 47, 40, 2, 34, 71, 16, 58, 35, 63, 72, 67, 59, 55, 6, 4, 30, 63, 52, 47, 52, 37, 56, 73, 17, 14, 71, 38, 25, 60, 1, 30, 54, 17, 20, 37, 59, 17, 11, 46, 44, 70, 36, 21, 65, 37, 7, 49, 63, 38, 66, 67, 6, 62, 43, 50, 35, 64, 41, 37, 0, 60, 24, 25, 59, 8, 59, 54, 10, 32, 57, 73, 60, 28, 67, 12, 0, 58, 37, 58, 36, 17, 38, 42, 32, 71, 27, 25, 60, 72, 50, 56, 59, 60, 31, 40, 50, 25, 22, 33, 9, 32, 52, 11, 60, 47, 7, 55, 66, 51, 69, 64, 47, 58, 73, 53, 8, 50, 27, 33, 36, 25, 40, 59, 23, 9, 29, 5, 70, 20, 55, 27, 39, 65, 55, 57, 9, 48, 32, 68, 4, 45, 18, 34, 35, 24, 7, 72, 35, 2, 29, 42, 39, 55, 44, 31, 60, 25, 30, 63, 36, 72, 19, 37, 49, 20, 27, 49, 14, 20, 67, 27, 13, 5, 4, 23, 66, 8, 41, 71, 47, 54, 28, 26, 15, 24, 41, 65, 47, 8, 67, 32, 22, 65, 56, 57, 67, 0, 69, 20, 19], 2096: [57, 6, 48, 58, 25, 62, 13, 55, 33, 2, 30, 11, 22, 18, 22, 46, 29, 22, 69, 69, 26, 35, 16, 24, 2, 6, 0, 30, 68, 73, 62, 28, 25, 19, 26, 32, 54, 48, 66, 67, 37, 8, 14, 1, 48, 12, 14, 46, 63, 63, 3, 68, 21, 70, 18, 51, 63, 4, 65, 12, 21, 1, 60, 27, 21, 28, 66, 57, 10, 65, 20, 38, 68, 53, 9, 28, 61, 71, 63, 38, 51, 29, 38, 6, 48, 42, 60, 50, 34, 21, 5, 38, 63, 67, 63, 52, 27, 52, 46, 49, 71, 15, 36, 73, 67, 57, 10, 8, 63, 1, 32, 18, 0, 20, 54, 3, 40, 27, 23, 44, 69, 20, 31, 14, 26, 42, 7, 62, 40, 65, 64, 54, 59, 66, 62, 57, 23, 71, 31, 36, 29, 10, 34, 33, 38, 73, 49, 48, 46, 35, 50, 12, 64, 59, 2, 25, 69, 64, 14, 14, 23, 22, 62, 65, 33, 46, 61, 1, 6, 18, 11, 36, 40, 65, 37, 3, 26, 20, 19, 61, 36, 54, 21, 1, 62, 23, 11, 31, 33, 18, 10, 3, 38, 30, 2, 20, 25, 69, 3, 26, 73, 59, 7, 5, 41, 52, 50, 14, 58, 43, 66, 21, 36, 55, 33, 66, 10, 61, 66, 58, 63, 13, 9, 31, 51, 5, 22, 52, 15, 21, 0, 16, 49, 34, 36, 35, 60, 17, 68, 49, 9, 41, 70, 64, 66, 1, 0, 9, 48, 54, 62, 8, 21, 3, 15, 30, 46, 10, 5, 29, 67, 4, 35, 66, 14, 2, 56, 73, 65, 13, 36, 30, 23, 46, 58, 37, 22, 52, 24, 66, 59, 2, 39, 52, 5, 52, 5, 7, 36, 20, 73, 52, 31, 66, 35, 3, 47, 3, 39, 4, 58, 5, 26, 47, 65, 35, 68, 62, 17, 17, 65, 3, 38, 59, 1, 71, 26, 61, 36, 11, 68, 14, 23, 55, 18, 28, 32, 53, 61, 56, 5, 18, 48, 63, 35, 49, 1, 16, 32, 59, 52, 62, 29, 72, 9, 57, 43, 29, 46, 38, 62, 12, 29, 52, 53, 73, 13, 3, 33, 37, 2, 63, 62, 32, 33, 27, 43, 66, 20, 71, 20, 54, 36, 72, 53, 57, 52, 63, 72, 49, 53, 6, 70, 15, 3, 28, 57, 56, 7, 67, 22, 49, 62, 70, 64, 62, 2, 31, 32, 32, 21, 55, 52, 62, 54, 30, 41, 56, 7, 32, 60, 47, 73, 16, 56, 7, 19, 26, 68, 20, 24, 50, 32, 26, 61, 73,
13, 3, 63, 20, 5, 44, 33, 31, 27, 26, 35, 60, 38, 29, 35, 13, 30, 63, 57, 34, 20, 52, 30, 16, 36, 47, 10, 58, 52, 67, 1, 9, 4, 4, 18, 31, 67, 68, 31, 20, 18, 65, 31, 19, 60, 16, 57, 26, 13, 10, 26, 64, 61, 63, 67, 15, 57, 26, 26], 1881: [58, 6, 72, 61, 27, 34, 7, 63, 66, 65, 47, 15, 1, 18, 22, 25, 29, 47, 10, 72, 69, 39, 16, 27, 68, 19, 28, 41, 68, 13, 5, 32, 27, 19, 26, 43, 54, 48, 58, 67, 27, 13, 12, 1, 46, 15, 14, 15, 63, 39, 2, 24, 22, 71, 18, 49, 42, 4, 65, 14, 26, 1, 28, 17, 62, 28, 66, 47, 34, 53, 67, 38, 66, 53, 34, 28, 60, 69, 72, 38, 49, 29, 38, 68, 48, 52, 57, 50, 34, 0, 5, 21, 63, 67, 63, 50, 21, 52, 48, 49, 71, 15, 1, 71, 67, 22, 7, 65, 61, 70, 20, 50, 69, 11, 44, 39, 34, 14, 56, 23, 24, 19, 29, 12, 47, 22, 12, 55, 0, 49, 25, 58, 60, 66, 68, 12, 20, 71, 7, 43, 50, 13, 59, 33, 72, 17, 29, 2, 49, 35, 50, 7, 68, 43, 6, 65, 18, 66, 12, 21, 23, 22, 62, 67, 69, 46, 51, 19, 6, 18, 34, 36, 57, 50, 36, 40, 15, 19, 39, 10, 36, 63, 19, 1, 62, 23, 11, 31, 66, 50, 68, 3, 43, 30, 59, 55, 1, 69, 34, 26, 38, 4, 37, 5, 39, 36, 54, 14, 7, 43, 64, 46, 60, 54, 33, 67, 10, 58, 60, 1, 63, 45, 52, 31, 51, 2, 11, 63, 18, 56, 36, 16, 49, 24, 41, 32, 60, 12, 72, 49, 11, 40, 70, 10, 37, 59, 2, 27, 5, 54, 60, 8, 66, 1, 17, 30, 47, 48, 16, 65, 61, 4, 34, 15, 62, 2, 66, 73, 65, 14, 2, 0, 48, 40, 69, 37, 64, 51, 55, 64, 57, 55, 39, 39, 3, 44, 8, 47, 50, 13, 60, 55, 30, 68, 4, 51, 36, 62, 39, 2, 30, 3, 26, 50, 22, 37, 5, 48, 20, 15, 69, 3, 28, 59, 1, 31, 6, 56, 35, 27, 68, 5, 23, 55, 44, 28, 32, 53, 59, 56, 44, 67, 46, 23, 6, 66, 24, 52, 33, 48, 52, 50, 3, 70, 0, 68, 43, 29, 48, 23, 26, 28, 29, 25, 53, 73, 18, 7, 33, 37, 2, 52, 72, 32, 36, 60, 38, 64, 32, 71, 27, 20, 50, 5, 54, 57, 67, 61, 64, 49, 55, 6, 7, 33, 7, 32, 55, 4, 8, 47, 7, 65, 66, 51, 64, 62, 70, 7, 32, 50, 62, 50, 39, 69, 36, 63, 42, 56, 72, 45, 22, 19, 54, 71, 56, 7, 20, 12, 68, 20, 38, 23, 47, 26, 38, 40, 54, 29, 68, 20, 5, 67, 41, 31, 27, 26, 35, 55, 38, 26, 63, 13, 30, 63, 36, 72, 20, 52, 49, 62, 36, 55, 14, 17, 54, 67, 0, 6, 4, 47, 71, 29, 8, 10, 5, 38, 18, 62, 6, 18, 69, 3, 21, 12, 42, 49, 27, 64, 56, 57, 68, 33, 57, 72, 19], 1978: [56, 72, 48, 14, 27, 53, 7, 62, 33, 65, 40, 15, 22, 19, 21, 25, 29, 22, 19, 73, 28, 40, 4, 28, 45, 4, 46, 30, 44, 19, 15, 28, 28, 19, 26, 46, 54, 48, 58, 69, 28, 8, 13, 0, 47, 12, 13, 73, 63, 67, 3, 54, 40, 22, 27, 52, 41, 3, 65, 50, 25, 26, 27, 31, 63, 47, 66, 70, 48, 69, 19, 36, 65, 53, 37, 28, 63, 26, 63, 39, 50, 19, 39, 51, 48, 53, 55, 40, 26, 21, 33, 22, 62, 66, 61, 51, 47, 54, 49, 69, 71, 15, 0, 72, 66, 18, 8, 66, 34, 1, 60, 54, 69, 40, 54, 3, 40, 27, 51, 69, 9, 20, 26, 1, 73, 42, 11, 47, 30, 50, 63, 68, 53, 7, 69, 13, 21, 50, 24, 2, 29, 14, 60, 54, 73, 18, 28, 20, 55, 35, 64, 7, 55, 63, 39, 25, 30, 22, 13, 13, 23, 21, 66, 14, 28, 43, 49, 20, 23, 18, 33, 35, 41, 61, 37, 68, 16, 19, 12, 11, 35, 63, 20, 0, 62, 21, 9, 59, 33, 51, 33, 3, 44, 28, 9, 22, 2, 69, 2, 24, 44, 1, 37, 58, 41, 37, 54, 1, 37, 43, 63, 42, 37, 55, 37, 53, 10, 60, 20, 57, 28, 61, 19, 32, 51, 2, 26, 13, 15, 52, 49, 7, 49, 63, 34, 33, 59, 10, 72, 55, 13, 61, 30, 10, 35, 54, 17, 9, 39, 50, 53, 72, 65, 5, 15, 30, 46, 49, 12, 6, 36, 4, 35, 16, 14, 2, 55, 73, 65, 13, 2, 0, 23, 40, 45, 36, 2, 52, 53, 66, 73, 25, 39, 50, 3, 43, 40, 7, 36, 10, 57, 55, 7, 66, 5, 7, 30, 33, 7, 4, 40, 4, 52, 49, 52, 37, 56, 49, 21, 16, 63, 3, 17, 58, 1, 30, 55, 55, 36, 11, 70, 14, 22, 55, 44, 28, 32, 52, 67, 58, 3, 68, 24, 67, 12, 72, 10, 63, 47, 10, 3, 64, 4, 41, 9, 56, 56, 29, 59, 37, 60, 28, 29, 65, 8, 73, 19, 7, 53, 35, 3, 61, 37, 33, 37, 30, 41, 65, 20, 71, 24, 57, 35, 72, 51, 56, 18, 62, 65, 48, 51, 25, 65, 33, 9, 32, 52, 4, 6, 58, 22, 53, 66, 17, 64, 62, 9, 7, 3, 51, 63, 55, 40, 33, 34, 64, 40, 67, 2, 9, 70, 19, 73, 16, 55, 9, 19, 12, 69, 21, 72, 3, 32, 26, 23, 73, 21, 27, 67, 20, 13, 73, 66, 72, 34, 39, 55, 55, 42, 24, 61, 12, 30, 63, 37, 73, 19, 51, 49, 16, 36, 50, 9, 17, 7, 59, 1, 7, 4, 3, 68, 31, 54, 72, 44, 37, 18, 7, 6, 19, 33, 66, 55, 12, 62, 1, 25, 65, 26, 58, 67, 33, 69, 24, 20], 2141: [59, 6, 73, 60, 25, 12, 8, 31, 33, 40, 39, 9, 27, 18, 35, 46, 29, 22, 69, 72, 4, 40, 3, 27, 0, 4, 45, 72, 43, 73, 20, 28, 27, 2, 26, 45, 54, 48, 39, 72, 24, 12, 21, 1, 46, 36, 13, 73, 62, 40, 3, 68, 23, 21, 23, 50, 32, 3, 65, 19, 23, 1, 25, 18, 64, 46, 53, 25, 47, 54, 18, 37, 66, 53, 7, 28, 59, 71, 22, 38, 49, 1, 38, 50, 53, 53, 58, 41, 34, 21, 7, 57, 62, 67, 63, 55, 22, 52, 48, 52, 2, 40, 0, 71, 65, 1, 38, 66, 66, 51, 56, 63, 71, 20, 45, 40, 34, 68, 57, 71, 1, 36, 30, 13, 36, 42, 52, 60, 32, 43, 24, 58, 59, 55, 61, 23, 25, 69, 5, 44, 25, 13, 2, 18, 73, 17, 73, 43, 44, 49, 50, 10, 67, 17, 21, 65, 0, 66, 12, 13, 24, 20, 43, 38, 70, 45, 59, 24, 7, 31, 35, 36, 62, 45, 15, 59, 15, 20, 17, 11, 29, 4, 45, 1, 64, 25, 56, 31, 5, 55, 33, 3, 43, 27, 57, 23, 6, 69, 39, 26, 38, 4, 38, 4, 39, 37, 53, 14, 8, 43, 66, 43, 1, 63, 18, 66, 4, 59, 21, 2, 65, 53, 53, 31, 50, 29, 8, 50, 23, 19, 5, 49, 48, 9, 61, 32, 59, 11, 73, 56, 16, 41, 60, 11, 38, 4, 39, 9, 48, 3, 60, 52, 66, 1, 15, 30, 46, 50, 14, 19, 61, 70, 33, 66, 63, 2, 29, 73, 65, 14, 15, 3, 38, 44, 4, 35, 2, 52, 24, 65, 59, 54, 39, 39, 5, 13, 5, 1, 50, 13, 71, 35, 31, 64, 40, 53, 45, 49, 39, 2, 56, 4, 25, 50, 34, 41, 68, 53, 20, 15, 70, 66, 18, 59, 29, 31, 64, 58, 8, 32, 68, 12, 21, 54, 62, 28, 0, 53, 68, 57, 44, 67, 66, 23, 35, 66, 4, 61, 34, 41, 52, 63, 68, 40, 1, 56, 45, 16, 64, 36, 61, 27, 29, 64, 69, 72, 59, 20, 71, 34, 2, 61, 61, 34, 41, 61, 1, 65, 60, 63, 28, 6, 35, 10, 38, 55, 67, 61, 65, 25, 51, 23, 6, 29, 12, 2, 33, 4, 7, 24, 20, 68, 33, 38, 72, 60, 4, 7, 9, 18, 20, 55, 32, 42, 52, 21, 41, 56, 65, 46, 70, 47, 73, 15, 56, 23, 20, 25, 68, 20, 13, 24, 47, 25,
 60, 2, 22, 1, 63, 22, 13, 68, 38, 57, 32, 38, 35, 56, 45, 27, 62, 11, 29, 64, 27, 73, 20, 51, 28, 16, 26, 16, 8, 56, 22, 58, 16, 8, 4, 46, 67, 30, 5, 60, 6, 37, 17, 63, 31, 19, 56, 73, 22, 13, 21, 49, 26, 64, 28, 51, 68, 32, 0, 23, 8], 2100: [27, 18, 66, 27, 16, 34, 40, 32, 11, 28, 30, 37, 45, 35, 22, 67, 11, 55, 10, 69, 11, 49, 24, 24, 19, 73, 47, 43, 68, 71, 62, 29, 37, 19, 42, 32, 67, 38, 58, 5, 48, 22, 14, 1, 35, 71, 70, 11, 16, 63, 46, 63, 46, 58, 24, 26, 42, 4, 65, 12, 4, 26, 48, 27, 7, 59, 62, 1, 62, 65, 20, 73, 53, 53, 6, 28, 60, 0, 12, 35, 51, 69, 51, 6, 61, 29, 72, 66, 13, 23, 5, 49, 8, 52, 14, 52, 23, 0, 45, 70, 39, 15, 36, 15, 67, 57, 19, 8, 68, 55, 57, 36, 69, 53, 55, 18, 64, 42, 10, 44, 10, 5, 29, 9, 3, 55, 12, 50, 0, 44, 65, 71, 53, 66, 63, 22, 5, 51, 25, 3, 0, 10, 61, 56, 73, 18, 29, 22, 44, 50, 4, 7, 55, 59, 2, 30, 69, 19, 72, 9, 33, 48, 62, 17, 56, 56, 61, 1, 24, 67, 54, 73, 42, 66, 54, 3, 21, 20, 20, 61, 45, 59, 32, 42, 27, 64, 40, 63, 61, 64, 68, 3, 15, 30, 26, 24, 71, 69, 24, 63, 45, 12, 7, 41, 6, 69, 50, 49, 43, 17, 62, 25, 36, 54, 25, 55, 10, 39, 66, 24, 35, 45, 54, 47, 51, 51, 27, 47, 42, 20, 24, 46, 18, 29, 41, 27, 60, 28, 5, 55, 53, 40, 0, 59, 63, 59, 22, 47, 39, 51, 58, 8, 61, 17, 15, 13, 47, 1, 16, 70, 33, 4, 10, 58, 11, 2, 56, 41, 65, 22, 61, 0, 48, 40, 47, 37, 11, 51, 55, 22, 11, 64, 40, 66, 3, 39, 13, 2, 36, 14, 39, 52, 11, 45, 17, 12, 64, 2, 39, 4, 57, 68, 26, 60, 12, 42, 56, 38, 56, 62, 17, 62, 21, 72, 6, 30, 26, 17, 3, 11, 53, 5, 4, 51, 49, 44, 32, 49, 61, 46, 63, 54, 17, 62, 66, 7, 11, 16, 32, 60, 31, 62, 5, 58, 57, 57, 24, 11, 60, 32, 60, 54, 45, 47, 57, 73, 30, 16, 52, 37, 0, 18, 56, 58, 27, 27, 4, 66, 34, 71, 27, 53, 47, 64, 10, 13, 64, 64, 33, 49, 63, 39, 7, 21, 10, 27, 52, 11, 66, 61, 27, 15, 69, 70, 39, 62, 14, 26, 32, 25, 12, 56, 72, 37, 41, 27, 42, 30, 25, 30, 24, 5, 4, 38, 21, 35, 42, 65, 63, 37, 73, 65, 34, 61, 61, 3, 63, 21, 35, 9, 5, 44, 33, 20, 27, 15, 35, 36, 38, 11, 63, 25, 30, 63, 68, 31, 65, 52, 28, 17, 41, 55, 10, 46, 13, 64, 51, 58, 26, 33, 10, 52, 44, 68, 12, 20, 72, 35, 1, 38, 37, 43, 52, 26, 70, 15, 27, 66, 1, 52, 25, 2, 57, 26, 21], 1957: [13, 10, 48, 14, 29, 16, 7, 43, 23, 65, 58, 15, 22, 19, 53, 26, 26, 26, 19, 73, 28, 40, 20, 28, 69, 4, 10, 34, 42, 17, 5, 18, 18, 23, 38, 49, 58, 10, 56, 69, 0, 18, 29, 1, 47, 16, 13, 55, 63, 67, 43, 27, 40, 54, 27, 71, 41, 46, 14, 16, 25, 2, 9, 65, 63, 24, 58, 70, 45, 69, 67, 36, 65, 39, 35, 28, 61, 26, 19, 39, 50, 32, 12, 7, 48, 53, 55, 53, 26, 19, 33, 59, 62, 8, 25, 48, 47, 18, 40, 73, 71, 19, 0, 69, 69, 18, 8, 66, 20, 41, 24, 67, 69, 22, 54, 3, 38, 27, 51, 69, 36, 17, 71, 70, 73, 23, 11, 51, 36, 40, 24, 68, 57, 7, 59, 29, 21, 50, 29, 43, 29, 14, 65, 54, 46, 18, 12, 48, 53, 47, 64, 7, 68, 63, 43, 25, 36, 68, 13, 18, 23, 62, 66, 67, 66, 67, 51, 70, 31, 28, 6, 28, 31, 5, 5, 68, 17, 19, 16, 2, 40, 63, 17, 0, 62, 21, 62, 64, 31, 51, 33, 51, 43, 26, 9, 55, 13, 35, 63, 30, 44, 59, 11, 58, 42, 65, 60, 1, 49, 9, 16, 21, 37, 51, 37, 68, 14, 28, 61, 55, 15, 61, 59, 50, 51, 2, 12, 13, 19, 52, 37, 62, 49, 63, 34, 33, 27, 10, 32, 55, 13, 47, 6, 9, 35, 54, 62, 65, 37, 60, 40, 22, 65, 3, 21, 28, 68, 63, 12, 66, 69, 4, 47, 61, 14, 72, 67, 73, 69, 15, 0, 0, 23, 41, 69, 72, 2, 52, 53, 64, 50, 67, 39, 31, 3, 43, 9, 48, 34, 10, 28, 24, 36, 68, 9, 7, 34, 33, 7, 4, 28, 52, 71, 47, 52, 38, 65, 49, 21, 14, 63, 3, 17, 60, 1, 32, 4, 1, 21, 28, 59, 18, 41, 55, 44, 26, 72, 45, 57, 56, 61, 68, 28, 67, 6, 0, 10, 63, 47, 54, 36, 64, 1, 38, 1, 7, 67, 29, 59, 9, 60, 26, 10, 33, 44, 73, 19, 7, 41, 33, 56, 32, 35, 7, 19, 58, 21, 65, 3, 71, 24, 13, 51, 6, 70, 58, 67, 61, 65, 36, 59, 27, 65, 33, 7, 6, 52, 4, 6, 58, 19, 53, 66, 51, 17, 1, 9, 9, 23, 51, 63, 51, 28, 33, 73, 23, 35, 67, 11, 9, 25, 19, 71, 12, 56, 7, 19, 63, 69, 21, 28, 3, 32, 26, 14, 8, 19, 42, 67, 31, 9, 73, 37, 72, 34, 39, 35, 66, 42, 24, 61, 12, 30, 63, 37, 73, 21, 47, 12, 21, 24, 51, 7, 64, 69, 57, 54, 7, 30, 24, 66, 4, 52, 72, 44, 18, 18, 7, 7, 19, 33, 66, 45, 12, 62, 1, 23, 65, 57, 58, 67, 33, 18, 22, 17], 2111: [12, 71, 46, 27, 28, 26, 39, 59, 6, 65, 42, 37, 22, 18, 27, 66, 29, 30, 15, 69, 29, 44, 3, 29, 43, 6, 43, 30, 71, 16, 56, 17, 17, 22, 37, 49, 59, 10, 61, 70, 47, 17, 19, 0, 51, 20, 3, 73, 53, 66, 46, 63, 39, 31, 5, 51, 41, 4, 62, 56, 29, 1, 55, 22, 73, 46, 66, 66, 37, 68, 23, 72, 61, 48, 2, 33, 62, 0, 68, 38, 67, 1, 11, 6, 61, 28, 26, 52, 9, 19, 5, 44, 6, 51, 63, 47, 23, 73, 40, 70, 72, 29, 54, 69, 65, 58, 38, 7, 63, 70, 52, 63, 69, 16, 54, 71, 37, 16, 51, 26, 69, 20, 14, 0, 1, 0, 10, 73, 35, 38, 64, 72, 56, 66, 73, 12, 10, 50, 25, 44, 25, 10, 61, 18, 69, 17, 29, 7, 52, 38, 7, 7, 55, 64, 39, 30, 58, 18, 16, 0, 42, 27, 9, 14, 71, 56, 50, 19, 27, 65, 24, 36, 2, 62, 41, 8, 20, 15, 8, 56, 55, 54, 35, 1, 21, 63, 66, 63, 30, 24, 73, 4, 61, 27, 8, 21, 71, 70, 19, 26, 40, 29, 33, 36, 39, 62, 70, 14, 53, 20, 12, 26, 37, 59, 33, 55, 13, 57, 61, 58, 31, 50, 51, 31, 54, 50, 22, 46, 20, 73, 3, 19, 14, 24, 36, 35, 72, 22, 68, 54, 12, 63, 5, 59, 68, 58, 39, 47, 5, 50, 53, 29, 0, 26, 22, 28, 47, 10, 12, 65, 61, 3, 10, 30, 63, 23, 55, 8, 70, 14, 40, 69, 48, 40, 61, 71, 7, 52, 54, 17, 73, 71, 39, 30, 5, 39, 40, 1, 50, 70, 27, 1, 11, 68, 69, 3, 30, 0, 39, 2, 58, 63, 53, 48, 51, 37, 64, 62, 22, 20, 71, 3, 16, 72, 1, 31, 55, 55, 6, 11, 71, 15, 23, 51, 44, 69, 33, 49, 56, 38, 6, 13, 27, 63, 44, 8, 6, 62, 46, 55, 2, 0, 46, 40, 11, 65, 57, 26, 60, 28, 58, 29, 10, 52, 58, 72, 35, 17, 66, 34, 33, 36, 38, 35, 33, 30, 1, 61, 21, 68, 35, 57, 46, 10, 54, 57, 67, 64, 64, 49, 55, 24, 12, 16, 9, 31, 51, 6, 9, 62, 23, 66, 63, 16, 69, 63, 8, 36, 22, 52, 8, 51, 35, 33, 36, 24, 42, 16, 10, 8, 21, 1, 70, 54, 30, 10, 42, 64, 68, 20,
  26, 53, 35, 25, 60, 73, 59, 34, 68, 20, 10, 39, 36, 57, 30, 31, 35, 71, 41, 11, 62, 23, 59, 23, 41, 28, 20, 51, 16, 12, 36, 50, 10, 17, 54, 67, 54, 58, 30, 33, 18, 29, 41, 71, 7, 53, 17, 8, 6, 14, 7, 38, 57, 8, 66, 10, 27, 64, 22, 47, 68, 32, 66, 21, 24], 1936: [11, 70, 46, 60, 28, 61, 11, 60, 6, 55, 39, 15, 22, 17, 23, 50, 25, 29, 15, 69, 28, 44, 19, 23, 43, 6, 44, 38, 70, 12, 57, 51, 16, 19, 72, 49, 58, 58, 55, 67, 47, 17, 17, 1, 46, 14, 18, 42, 53, 65, 3, 52, 38, 22, 25, 50, 43, 49, 64, 56, 28, 0, 25, 18, 72, 42, 57, 38, 36, 67, 19, 34, 63, 53, 1, 28, 61, 58, 73, 37, 53, 31, 37, 72, 58, 29, 67, 52, 43, 19, 1, 43, 7, 51, 4, 2, 51, 73, 40, 65, 72, 64, 54, 68, 64, 59, 18, 7, 18, 51, 20, 50, 68, 22, 54, 36, 36, 15, 53, 39, 71, 16, 14, 0, 73, 13, 11, 51, 28, 38, 64, 68, 55, 65, 72, 72, 10, 71, 25, 2, 29, 10, 39, 33, 72, 16, 54, 19, 41, 45, 62, 7, 55, 15, 60, 65, 69, 66, 12, 0, 41, 22, 62, 67, 31, 46, 52, 69, 31, 66, 71, 36, 39, 61, 4, 12, 15, 19, 14, 42, 36, 44, 36, 1, 60, 23, 39, 60, 8, 51, 65, 3, 57, 30, 7, 20, 71, 69, 62, 26, 40, 29, 11, 36, 40, 61, 58, 0, 8, 39, 12, 25, 37, 59, 32, 51, 14, 56, 61, 58, 62, 49, 21, 30, 50, 9, 22, 46, 19, 64, 3, 60, 49, 4, 36, 56, 72, 9, 68, 53, 8, 28, 6, 7, 68, 32, 26, 47, 36, 50, 31, 71, 25, 1, 17, 29, 47, 9, 1, 31, 65, 3, 19, 19, 16, 2, 55, 73, 69, 13, 40, 42, 19, 44, 65, 70, 2, 48, 52, 35, 73, 70, 39, 65, 4, 39, 40, 7, 49, 70, 26, 0, 6, 68, 8, 66, 33, 5, 39, 2, 58, 4, 53, 45, 52, 37, 64, 73, 21, 19, 70, 73, 16, 60, 1, 60, 7, 55, 36, 11, 71, 2, 70, 53, 44, 69, 32, 21, 65, 58, 6, 5, 62, 23, 44, 66, 6, 61, 42, 54, 34, 62, 0, 37, 11, 65, 57, 25, 60, 58, 58, 29, 28, 29, 54, 72, 15, 11, 62, 33, 1, 13, 38, 34, 35, 30, 39, 61, 22, 67, 35, 57, 54, 72, 55, 9, 63, 60, 13, 71, 55, 26, 2, 33, 8, 2, 51, 6, 9, 47, 24, 65, 66, 15, 68, 63, 7, 57, 23, 73, 7, 51, 40, 32, 36, 68, 43, 56, 72, 9, 19, 19, 54, 54, 55, 39, 39, 9, 44, 19, 26, 52, 34, 12, 56, 44, 50, 30, 67, 21, 5, 68, 33, 56, 27, 37, 35, 14, 40, 11, 40, 13, 58, 23, 65, 35, 20, 54, 15, 20, 36, 49, 11, 68, 54, 26, 36, 5, 3, 60, 71, 7, 4, 70, 6, 16, 67, 67, 6, 19, 32, 32, 22, 10, 60, 49, 22, 64, 25, 14, 67, 31, 69, 28, 16], 2138: [13, 36, 72, 25, 24, 11, 22, 29, 0, 68, 34, 9, 26, 73, 26, 69, 27, 0, 53, 71, 30, 48, 5, 26, 59, 72, 46, 40, 53, 17, 66, 20, 54, 23, 5, 36, 19, 0, 58, 71, 31, 19, 22, 3, 47, 19, 59, 41, 55, 67, 1, 66, 40, 24, 22, 23, 45, 53, 63, 50, 14, 26, 54, 47, 6, 47, 59, 42, 67, 69, 18, 70, 65, 52, 5, 32, 65, 26, 72, 39, 48, 33, 12, 0, 60, 40, 56, 25, 24, 51, 54, 63, 7, 66, 25, 50, 45, 20, 67, 11, 0, 17, 53, 70, 66, 61, 22, 7, 67, 41, 63, 61, 69, 1, 43, 7, 38, 20, 55, 3, 9, 18, 25, 54, 73, 42, 11, 47, 32, 42, 63, 68, 57, 4, 15, 21, 13, 41, 24, 24, 31, 12, 32, 54, 72, 20, 16, 19, 43, 49, 64, 7, 55, 17, 62, 40, 69, 18, 14, 35, 23, 21, 64, 16, 68, 59, 54, 71, 21, 66, 1, 71, 41, 42, 6, 40, 23, 40, 16, 31, 44, 46, 36, 46, 26, 13, 43, 42, 53, 28, 67, 72, 30, 26, 9, 22, 65, 33, 61, 10, 58, 15, 37, 47, 42, 66, 61, 1, 51, 42, 16, 42, 35, 8, 71, 3, 14, 58, 15, 55, 35, 1, 52, 32, 51, 2, 59, 50, 45, 56, 6, 62, 49, 23, 53, 31, 30, 25, 72, 55, 8, 39, 15, 9, 37, 56, 21, 27, 39, 49, 56, 32, 20, 0, 13, 28, 72, 63, 12, 18, 17, 4, 31, 19, 18, 27, 55, 72, 7, 13, 35, 22, 37, 39, 65, 51, 16, 50, 53, 39, 72, 23, 0, 31, 17, 43, 42, 22, 34, 10, 59, 36, 8, 71, 10, 71, 45, 1, 41, 59, 62, 63, 55, 47, 52, 39, 56, 36, 19, 21, 73, 65, 17, 60, 3, 41, 30, 1, 2, 31, 59, 4, 72, 55, 44, 73, 70, 23, 67, 37, 61, 16, 26, 62, 10, 6, 8, 63, 68, 52, 36, 54, 2, 69, 0, 48, 56, 27, 59, 61, 60, 28, 38, 44, 57, 73, 60, 15, 5, 14, 48, 45, 37, 36, 37, 33, 0, 42, 24, 7, 27, 19, 18, 14, 51, 17, 67, 69, 63, 40, 59, 10, 5, 15, 11, 32, 34, 72, 6, 60, 26, 14, 66, 51, 73, 7, 9, 35, 5, 19, 11, 54, 49, 26, 44, 1, 40, 69, 72, 45, 21, 21, 72, 37, 32, 7, 41, 63, 46, 12, 20, 35, 32, 26, 4, 0, 58, 2, 3, 4, 7, 10, 66, 70, 59, 60, 39, 55, 42, 51, 61, 12, 62, 25, 67, 65, 19, 25, 49, 22, 40, 51, 14, 73, 7, 57, 54, 7, 30, 24, 66, 8, 6, 72, 29, 58, 34, 7, 6, 37, 54, 2, 60, 12, 69, 59, 24, 65, 25, 33, 71, 13, 56, 22, 19], 2065: [12, 9, 72, 61, 28, 62, 39, 59, 33, 65, 43, 15, 32, 58, 15, 50, 25, 47, 19, 13, 28, 44, 19, 27, 43, 23, 43, 30, 71, 16, 4, 51, 16, 23, 26, 49, 59, 48, 55, 68, 47, 17, 17, 0, 46, 37, 17, 41, 53, 67, 43, 63, 37, 56, 24, 12, 41, 41, 69, 22, 29, 1, 29, 30, 62, 32, 57, 66, 37, 20, 67, 35, 64, 48, 2, 29, 62, 0, 72, 39, 67, 31, 55, 68, 57, 28, 71, 52, 25, 19, 8, 44, 50, 66, 64, 2, 10, 53, 40, 52, 72, 15, 1, 68, 66, 22, 14, 65, 61, 50, 44, 53, 69, 22, 40, 36, 35, 26, 17, 26, 70, 16, 71, 20, 61, 22, 11, 30, 35, 38, 28, 9, 51, 22, 73, 17, 10, 67, 60, 39, 57, 13, 59, 34, 33, 17, 72, 2, 49, 38, 61, 7, 55, 62, 5, 65, 35, 67, 12, 31, 22, 18, 37, 67, 71, 46, 51, 69, 31, 27, 43, 27, 39, 41, 41, 40, 16, 15, 15, 43, 35, 63, 17, 0, 62, 23, 61, 51, 8, 50, 32, 2, 30, 25, 6, 21, 15, 70, 63, 26, 40, 58, 26, 36, 39, 64, 48, 17, 38, 39, 67, 26, 60, 59, 33, 51, 13, 57, 60, 54, 63, 67, 10, 31, 54, 9, 11, 2, 18, 22, 3, 10, 49, 24, 31, 55, 26, 9, 31, 52, 1, 15, 5, 59, 34, 53, 2, 8, 36, 6, 19, 71, 25, 4, 55, 28, 47, 48, 12, 65, 61, 4, 38, 66, 61, 3, 43, 3, 68, 17, 40, 42, 22, 47, 61, 41, 26, 53, 52, 64, 60, 70, 39, 20, 3, 47, 40, 47, 53, 70, 56, 58, 35, 68, 8, 67, 50, 19, 70, 3, 44, 3, 52, 46, 52, 37, 64, 73, 20, 14, 69, 37, 16, 60, 1, 60, 59, 1, 72, 27, 71, 14, 40, 40, 44, 69, 33, 21, 64, 55, 6, 1, 63, 23, 65, 70, 6, 62, 35, 49, 3, 51, 35, 4, 11, 60, 34, 6, 48, 26, 26, 28, 10, 32, 54, 73, 18, 7, 15, 11, 0, 40, 72, 6, 36, 30, 20, 57, 2, 67, 24, 57, 50, 1, 34, 9, 67, 60, 13, 40, 58, 26, 12, 33, 6, 32, 31, 6, 9, 47, 7, 53, 66, 16, 16, 1, 47, 7, 22, 50, 8, 50, 56, 33, 36, 67, 37, 56, 72, 9, 51, 1, 54, 55, 56, 11, 
  19, 9, 69, 20, 38, 53, 48, 12, 64, 44, 54, 31, 71, 21, 57, 72, 37, 57, 31, 42, 53, 55, 41, 23, 60, 12, 57, 62, 36, 26, 19, 54, 8, 20, 36, 49, 6, 20, 68, 27, 4, 6, 30, 47, 66, 7, 4, 71, 47, 16, 18, 6, 6, 57, 32, 65, 68, 12, 0, 49, 22, 65, 25, 14, 67, 33, 17, 48, 16], 2112: [56, 71, 48, 27, 27, 31, 43, 65, 33, 68, 29, 10, 44, 17, 21, 45, 33, 43, 54, 69, 28, 49, 6, 27, 44, 10, 44, 29, 44, 71, 61, 50, 19, 19, 26, 48, 62, 37, 57, 69, 50, 17, 18, 1, 46, 14, 70, 73, 62, 62, 45, 23, 40, 22, 4, 23, 43, 45, 65, 50, 2, 0, 22, 29, 7, 27, 19, 65, 47, 69, 19, 37, 65, 53, 33, 28, 65, 25, 2, 37, 56, 33, 11, 5, 48, 34, 70, 11, 24, 2, 4, 63, 7, 71, 4, 50, 27, 52, 44, 72, 71, 15, 26, 68, 64, 62, 17, 7, 67, 67, 69, 63, 71, 30, 44, 13, 36, 66, 72, 26, 68, 19, 26, 3, 73, 17, 10, 3, 32, 50, 63, 68, 53, 65, 73, 21, 9, 69, 24, 24, 54, 10, 64, 55, 73, 18, 28, 19, 44, 49, 64, 7, 55, 17, 39, 65, 69, 18, 12, 13, 23, 20, 64, 16, 68, 46, 49, 69, 31, 18, 32, 35, 62, 3, 4, 42, 15, 19, 7, 43, 38, 44, 20, 0, 64, 25, 63, 62, 6, 51, 67, 2, 71, 28, 29, 24, 7, 73, 36, 5, 38, 1, 11, 58, 41, 61, 64, 0, 52, 43, 60, 25, 35, 53, 35, 55, 5, 60, 66, 57, 62, 53, 21, 30, 51, 12, 24, 50, 17, 56, 18, 46, 49, 23, 38, 34, 55, 11, 67, 56, 42, 46, 23, 10, 68, 3, 2, 27, 38, 53, 60, 12, 65, 0, 14, 30, 47, 71, 15, 5, 61, 4, 31, 58, 62, 2, 55, 10, 65, 13, 2, 45, 22, 39, 58, 36, 26, 51, 54, 64, 72, 23, 38, 64, 4, 50, 40, 43, 49, 10, 71, 36, 35, 66, 4, 70, 46, 1, 41, 4, 57, 63, 20, 49, 50, 37, 66, 61, 19, 16, 64, 37, 25, 60, 1, 30, 64, 59, 34, 11, 70, 17, 22, 55, 44, 73, 31, 52, 69, 58, 62, 67, 66, 62, 26, 6, 10, 62, 47, 58, 6, 64, 49, 69, 0, 73, 56, 25, 59, 61, 61, 53, 28, 29, 57, 73, 59, 16, 32, 36, 33, 10, 37, 34, 32, 33, 42, 57, 19, 71, 34, 19, 62, 72, 56, 56, 66, 70, 66, 21, 60, 38, 64, 33, 9, 27, 34, 11, 7, 61, 26, 65, 66, 51, 64, 61, 9, 25, 31, 49, 30, 54, 32, 32, 39, 21, 40, 67, 72, 9, 21, 19, 72, 59, 55, 9, 19, 12, 67, 19, 26, 21, 32, 26, 37, 0, 58, 2, 35, 19, 7, 36, 35, 56, 29, 38, 71, 55, 21, 23, 62, 11, 28, 63, 27, 31, 19, 54, 28, 22, 35, 50, 9, 57, 8, 58, 0, 8, 3, 33, 27, 10, 7, 72, 5, 19, 35, 7, 6, 17, 33, 42, 59, 7, 70, 9, 24, 64, 25, 57, 67, 31, 56, 25, 30], 2072: [13, 12, 70, 60, 27, 11, 7, 60, 33, 65, 38, 15, 33, 17, 15, 26, 33, 0, 14, 69, 28, 39, 20, 24, 44, 20, 44, 29, 30, 12, 5, 50, 27, 19, 26, 45, 58, 48, 38, 69, 0, 17, 18, 1, 47, 14, 17, 72, 61, 39, 51, 23, 38, 22, 22, 52, 43, 5, 69, 18, 28, 0, 27, 17, 30, 8, 57, 61, 46, 50, 65, 36, 65, 53, 39, 28, 63, 26, 21, 39, 43, 31, 37, 5, 32, 29, 57, 42, 22, 1, 37, 40, 63, 66, 25, 48, 21, 52, 48, 51, 1, 15, 0, 68, 69, 59, 39, 66, 18, 66, 55, 62, 70, 22, 20, 39, 36, 64, 18, 70, 19, 16, 71, 52, 73, 22, 51, 3, 42, 38, 26, 68, 55, 21, 16, 13, 27, 73, 6, 43, 31, 10, 59, 35, 73, 18, 73, 2, 49, 37, 50, 7, 66, 63, 6, 65, 69, 66, 12, 33, 23, 24, 62, 67, 66, 46, 49, 69, 6, 28, 44, 35, 39, 49, 4, 40, 15, 19, 53, 65, 36, 3, 36, 0, 64, 52, 11, 30, 6, 51, 8, 3, 28, 26, 56, 22, 15, 69, 38, 25, 38, 59, 11, 58, 40, 61, 49, 18, 52, 43, 8, 42, 1, 62, 33, 51, 14, 58, 20, 55, 62, 52, 21, 32, 51, 2, 12, 13, 17, 56, 4, 20, 49, 8, 38, 34, 27, 10, 72, 53, 13, 46, 6, 19, 35, 3, 2, 27, 37, 64, 73, 71, 65, 1, 17, 26, 47, 9, 1, 31, 69, 2, 39, 72, 62, 2, 55, 4, 64, 15, 17, 0, 37, 44, 66, 73, 2, 51, 53, 64, 37, 67, 14, 29, 4, 15, 40, 73, 49, 70, 60, 63, 35, 65, 37, 66, 55, 5, 39, 4, 30, 1, 50, 49, 50, 37, 65, 34, 21, 14, 67, 4, 17, 58, 1, 30, 3, 1, 36, 28, 70, 16, 20, 55, 1, 61, 31, 21, 65, 58, 44, 14, 46, 23, 10, 66, 6, 58, 47, 47, 66, 64, 31, 37, 4, 72, 45, 25, 48, 36, 26, 26, 28, 29, 59, 73, 71, 13, 62, 33, 1, 60, 60, 34, 32, 60, 42, 21, 60, 67, 24, 53, 34, 1, 55, 56, 19, 60, 66, 21, 59, 27, 64, 33, 8, 30, 32, 6, 9, 47, 19, 65, 66, 37, 68, 5, 4, 7, 3, 73, 63, 45, 31, 64, 36, 62, 43, 67, 11, 9, 24, 19, 71, 11, 55, 9, 19, 63, 69, 19, 12, 3, 61, 26, 22, 6, 58, 30, 67, 4, 5, 73, 35, 56, 29, 27, 54, 55, 42, 26, 61, 13, 28, 63, 27, 18, 19, 54, 17, 20, 25, 49, 9, 57, 69, 57, 0, 7, 3, 47, 66, 29, 69, 27, 5, 39, 16, 63, 33, 19, 55, 65, 23, 12, 61, 53, 14, 63, 25, 57, 69, 31, 6, 20, 17], 2194: [10, 33, 50, 60, 27, 63, 7, 64, 22, 3, 40, 15, 50, 8, 23, 49, 30, 56, 54, 11, 25, 36, 19, 23, 42, 5, 46, 31, 69, 14, 4, 50, 0, 21, 5, 54, 29, 50, 55, 66, 45, 8, 19, 0, 45, 16, 47, 72, 58, 65, 54, 23, 38, 22, 5, 12, 43, 5, 66, 13, 25, 0, 51, 17, 72, 29, 66, 58, 49, 21, 19, 34, 67, 54, 8, 29, 60, 58, 73, 39, 45, 30, 39, 49, 49, 55, 67, 51, 36, 4, 6, 50, 47, 68, 6, 51, 28, 51, 50, 53, 72, 15, 16, 68, 64, 26, 19, 65, 34, 50, 52, 49, 20, 21, 44, 4, 33, 15, 26, 38, 71, 19, 71, 19, 73, 54, 9, 56, 1, 52, 26, 68, 61, 67, 71, 22, 10, 70, 8, 43, 50, 10, 11, 55, 73, 17, 53, 21, 48, 37, 51, 7, 65, 63, 3, 27, 19, 8, 11, 30, 41, 4, 42, 14, 69, 45, 55, 69, 8, 27, 35, 35, 41, 39, 39, 37, 36, 19, 13, 13, 28, 6, 21, 0, 63, 22, 12, 40, 7, 5, 34, 2, 59, 29, 4, 19, 15, 70, 61, 60, 40, 28, 32, 23, 40, 61, 62, 46, 72, 44, 65, 22, 37, 59, 20, 0, 0, 62, 20, 25, 64, 49, 8, 32, 50, 72, 22, 46, 19, 20, 3, 36, 49, 5, 36, 28, 27, 10, 69, 53, 12, 43, 32, 64, 73, 3, 19, 72, 4, 50, 53, 29, 67, 4, 16, 28, 47, 49, 10, 65, 59, 3, 37, 30, 16, 19, 17, 49, 66, 12, 40, 41, 37, 15, 46, 38, 26, 53, 25, 36, 58, 65, 17, 29, 4, 53, 43, 7, 37, 70, 57, 0, 30, 68, 39, 68, 54, 17, 38, 1, 60, 63, 52, 44, 52, 36, 6, 64, 36, 19, 70, 2, 40, 60, 0, 53, 56, 42, 36, 28, 70, 14, 22, 52, 3, 30, 31, 54, 71, 55, 5, 13, 26, 35, 14, 73, 5, 61, 33, 49, 53, 64, 38, 36, 1, 69, 45, 6, 25, 37, 57, 55, 28, 53, 54, 72, 19, 19, 34, 38, 4, 13, 71, 33, 35, 30, 33, 66, 70, 72, 24, 57, 6, 6, 53, 56, 64, 64, 63, 18, 54, 7, 28, 16, 8, 29, 32, 6, 9, 46, 23, 53, 63, 52, 73, 63, 7, 32, 33, 72, 6, 56, 40, 0, 35, 71, 41, 44, 2, 8, 60, 46, 73, 53,
   57, 11, 55, 24, 69, 19, 15, 51, 33, 49, 39, 3, 15, 29, 5, 21, 4, 73, 32, 42, 31, 37, 57, 52, 41, 3, 59, 13, 71, 22, 57, 35, 19, 53, 31, 63, 37, 47, 11, 18, 53, 60, 2, 58, 30, 5, 70, 33, 56, 9, 44, 39, 17, 66, 6, 19, 52, 65, 69, 12, 1, 12, 26, 63, 16, 55, 67, 31, 56, 28, 15], 2036: [56, 72, 48, 58, 25, 31, 41, 61, 33, 37, 39, 13, 47, 18, 21, 44, 28, 43, 69, 72, 26, 34, 3, 27, 44, 4, 44, 30, 44, 73, 64, 64, 40, 19, 26, 58, 27, 73, 57, 70, 43, 8, 19, 1, 46, 8, 45, 73, 62, 63, 43, 35, 39, 22, 5, 50, 43, 48, 65, 12, 23, 25, 25, 27, 73, 28, 19, 57, 47, 68, 19, 37, 66, 53, 33, 28, 61, 56, 37, 38, 56, 1, 11, 6, 47, 30, 58, 12, 25, 19, 38, 57, 46, 53, 26, 51, 22, 73, 45, 54, 2, 18, 14, 0, 65, 18, 17, 7, 48, 39, 32, 63, 71, 28, 44, 37, 39, 43, 15, 26, 69, 58, 26, 0, 1, 0, 10, 3, 30, 50, 64, 14, 59, 66, 73, 57, 73, 69, 25, 40, 25, 13, 64, 18, 61, 17, 28, 25, 55, 49, 50, 7, 55, 17, 39, 65, 69, 66, 12, 0, 21, 2, 60, 14, 33, 46, 56, 69, 6, 65, 32, 36, 62, 40, 17, 55, 15, 19, 14, 46, 55, 62, 19, 15, 62, 21, 58, 40, 7, 51, 68, 8, 57, 27, 8, 20, 57, 69, 39, 8, 73, 29, 7, 42, 41, 33, 50, 0, 53, 43, 63, 43, 36, 53, 69, 66, 16, 59, 66, 56, 62, 53, 21, 50, 50, 48, 22, 48, 43, 73, 18, 21, 47, 5, 36, 44, 59, 11, 68, 54, 43, 28, 69, 10, 71, 54, 17, 48, 5, 54, 55, 11, 66, 1, 15, 30, 46, 10, 10, 31, 7, 3, 62, 66, 14, 3, 55, 59, 65, 11, 2, 45, 69, 40, 56, 71, 53, 52, 54, 65, 16, 23, 39, 67, 5, 39, 5, 44, 35, 9, 57, 51, 6, 66, 20, 51, 32, 7, 37, 2, 58, 69, 53, 50, 51, 37, 55, 62, 20, 17, 65, 63, 71, 59, 1, 61, 64, 55, 4, 11, 68, 17, 21, 55, 44, 28, 32, 52, 69, 55, 3, 67, 24, 62, 12, 71, 42, 62, 32, 59, 2, 0, 49, 40, 34, 73, 2, 28, 60, 61, 61, 53, 26, 50, 65, 72, 17, 17, 33, 36, 2, 13, 61, 34, 32, 28, 1, 63, 68, 71, 35, 32, 46, 72, 56, 57, 67, 61, 61, 5, 60, 41, 52, 24, 9, 28, 29, 11, 7, 58, 64, 21, 66, 16, 64, 60, 8, 5, 32, 38, 31, 55, 32, 32, 39, 69, 66, 19, 72, 32, 21, 19, 54, 59, 55, 10, 20, 64, 58, 20, 24, 22, 59, 22, 60, 73, 13, 3, 3, 27, 5, 16, 38, 42, 24, 37, 72, 66, 43, 31, 62, 11, 29, 20, 23, 32, 20, 51, 48, 10, 36, 50, 9, 19, 54, 58, 1, 8, 4, 61, 67, 31, 54, 71, 6, 17, 17, 8, 6, 39, 52, 0, 60, 10, 7, 10, 26, 64, 4, 53, 58, 32, 57, 47, 26], 2063: [13, 72, 50, 27, 25, 31, 41, 63, 10, 65, 37, 9, 46, 15, 22, 67, 27, 56, 69, 69, 26, 46, 3, 29, 55, 6, 43, 30, 54, 71, 63, 17, 17, 19, 26, 51, 70, 35, 58, 67, 45, 8, 19, 1, 46, 10, 48, 73, 16, 61, 1, 54, 41, 58, 49, 21, 42, 4, 64, 51, 5, 24, 23, 27, 73, 56, 66, 57, 45, 69, 18, 34, 66, 53, 31, 28, 57, 71, 63, 38, 51, 69, 9, 6, 61, 30, 72, 22, 13, 19, 5, 21, 9, 51, 15, 50, 45, 73, 46, 49, 72, 15, 17, 16, 67, 58, 19, 8, 69, 65, 32, 59, 69, 20, 43, 39, 38, 45, 25, 69, 71, 18, 26, 9, 73, 42, 12, 15, 30, 21, 64, 68, 55, 66, 12, 19, 10, 51, 25, 3, 50, 10, 34, 54, 73, 17, 49, 20, 41, 67, 50, 7, 55, 64, 19, 32, 30, 16, 11, 12, 21, 18, 62, 14, 33, 44, 54, 2, 5, 67, 72, 36, 42, 40, 50, 3, 24, 17, 14, 45, 36, 63, 36, 42, 62, 23, 40, 40, 33, 53, 68, 3, 69, 30, 9, 20, 66, 69, 24, 60, 73, 12, 37, 37, 39, 59, 62, 14, 48, 43, 62, 26, 37, 51, 17, 66, 10, 58, 20, 58, 35, 51, 17, 50, 51, 48, 22, 47, 43, 17, 48, 61, 47, 41, 36, 32, 72, 9, 65, 56, 11, 14, 24, 61, 68, 54, 17, 48, 5, 50, 53, 29, 59, 3, 15, 28, 46, 10, 13, 6, 59, 4, 10, 33, 16, 2, 56, 73, 65, 13, 61, 43, 48, 40, 56, 38, 13, 51, 54, 17, 73, 23, 39, 67, 3, 3, 5, 19, 34, 8, 27, 53, 8, 68, 69, 73, 31, 7, 39, 4, 60, 69, 53, 47, 12, 37, 56, 33, 33, 22, 14, 3, 25, 57, 9, 31, 55, 56, 32, 11, 68, 15, 23, 52, 44, 71, 33, 50, 68, 43, 5, 10, 64, 63, 14, 72, 1, 61, 29, 61, 4, 62, 46, 70, 7, 68, 5, 4, 60, 34, 61, 12, 27, 53, 60, 73, 60, 19, 2, 34, 31, 13, 38, 33, 33, 30, 1, 65, 21, 71, 27, 30, 6, 72, 51, 19, 64, 61, 64, 49, 51, 7, 6, 33, 11, 28, 31, 11, 7, 58, 66, 66, 66, 16, 70, 62, 10, 5, 23, 52, 63, 55, 72, 60, 41, 30, 42, 38, 72, 30, 21, 9, 73, 56, 33, 7, 42, 24, 44, 18, 24, 66, 34, 24, 35, 73, 20, 36, 4, 21, 5, 64, 33, 57, 27, 36, 35, 68, 38, 12, 62, 13, 30, 63, 44, 34, 20, 22, 46, 12, 36, 52, 11, 17, 7, 64, 55, 58, 30, 4, 24, 8, 41, 72, 5, 18, 34, 62, 6, 18, 41, 2, 49, 12, 0, 12, 27, 66, 26, 51, 68, 33, 57, 27, 26], 2164: [13, 33, 50, 27, 28, 30, 39, 64, 6, 39, 41, 15, 45, 8, 24, 67, 26, 57, 19, 69, 28, 44, 19, 29, 43, 23, 44, 41, 72, 17, 57, 51, 16, 19, 26, 67, 32, 1, 38, 69, 0, 17, 20, 1, 47, 19, 48, 42, 53, 65, 46, 53, 38, 54, 25, 50, 43, 46, 62, 28, 9, 0, 25, 72, 73, 51, 57, 51, 38, 68, 19, 73, 65, 48, 39, 28, 70, 58, 73, 39, 53, 33, 10, 7, 61, 29, 26, 26, 24, 4, 37, 45, 7, 66, 25, 2, 45, 72, 40, 71, 72, 15, 40, 68, 64, 59, 19, 7, 52, 66, 61, 62, 70, 12, 54, 15, 15, 42, 18, 1, 38, 16, 14, 0, 73, 0, 11, 45, 33, 38, 64, 68, 55, 67, 16, 17, 11, 50, 25, 3, 58, 13, 62, 55, 73, 18, 29, 20, 53, 45, 62, 7, 56, 63, 60, 17, 69, 19, 15, 33, 23, 19, 62, 16, 56, 57, 55, 69, 31, 66, 6, 35, 39, 42, 6, 12, 21, 19, 16, 43, 40, 54, 36, 46, 64, 62, 39, 61, 8, 51, 37, 3, 72, 26, 7, 22, 45, 69, 38, 60, 40, 2, 32, 37, 40, 64, 70, 1, 52, 40, 20, 25, 37, 59, 37, 51, 14, 58, 61, 55, 30, 1, 21, 32, 50, 2, 58, 47, 19, 73, 3, 62, 13, 24, 38, 31, 72, 10, 32, 53, 8, 63, 6, 10, 68, 57, 19, 47, 37, 50, 54, 29, 25, 25, 14, 28, 69, 12, 13, 65, 37, 3, 32, 31, 16, 19, 55, 73, 73, 13, 41, 0, 37, 37, 69, 41, 11, 51, 53, 37, 73, 71, 39, 65, 4, 39, 40, 23, 34, 70, 26, 63, 6, 68, 8, 12, 34, 5, 39, 4, 58, 63, 53, 47, 52, 37, 65, 34, 21, 19, 67, 73, 17, 61, 1, 60, 55, 1, 36, 11, 59, 14, 70, 44, 44, 30, 33, 21, 65, 43, 6, 67, 26, 63, 44, 7, 6, 61, 42, 54, 34, 62, 2, 39, 11, 49, 57, 7, 60, 32, 58, 26, 28, 52, 57, 72, 19, 11, 16, 33, 1, 14, 38, 34, 35, 31, 0, 63, 22, 6, 24, 39, 51, 12, 2, 9, 67, 65, 13, 40, 59, 27, 12, 16, 8, 32, 35, 51, 9, 62, 24, 65, 66, 15, 17, 63, 7, 57, 22, 51, 12, 51, 40, 32, 36, 23,
    43, 34, 72, 9, 52, 21, 70, 54, 33, 11, 39, 63, 55, 19, 26, 52, 34, 68, 4, 44, 58, 30, 1, 21, 9, 73, 64, 73, 57, 42, 35, 55, 42, 11, 61, 23, 71, 22, 49, 36, 19, 54, 47, 20, 36, 49, 11, 68, 54, 68, 55, 59, 30, 24, 66, 6, 4, 71, 17, 16, 33, 8, 6, 19, 33, 65, 21, 12, 65, 10, 22, 66, 26, 14, 67, 31, 71, 22, 17], 1904: [11, 71, 71, 27, 28, 62, 9, 58, 7, 56, 42, 15, 49, 18, 22, 50, 26, 55, 15, 69, 28, 35, 3, 24, 58, 5, 25, 30, 45, 13, 5, 51, 28, 19, 73, 45, 59, 48, 60, 70, 45, 18, 18, 1, 47, 36, 38, 73, 53, 63, 55, 63, 36, 23, 5, 51, 41, 4, 65, 50, 29, 26, 26, 27, 73, 59, 66, 62, 47, 68, 18, 73, 65, 53, 33, 28, 65, 71, 2, 38, 51, 69, 11, 6, 61, 33, 69, 52, 36, 5, 5, 44, 48, 73, 6, 52, 23, 73, 40, 52, 71, 15, 1, 69, 16, 61, 19, 67, 66, 67, 52, 63, 69, 20, 54, 13, 37, 16, 52, 26, 70, 17, 27, 0, 1, 42, 10, 47, 47, 39, 64, 68, 53, 66, 73, 23, 10, 37, 25, 3, 59, 10, 61, 18, 31, 17, 29, 21, 49, 49, 4, 7, 55, 64, 39, 66, 70, 67, 13, 14, 42, 26, 43, 16, 68, 56, 50, 70, 32, 28, 29, 36, 40, 42, 5, 31, 16, 19, 16, 66, 36, 45, 21, 42, 64, 53, 13, 61, 35, 52, 67, 4, 72, 28, 5, 23, 12, 69, 36, 26, 40, 2, 39, 22, 41, 62, 50, 1, 39, 43, 6, 25, 38, 60, 69, 49, 10, 57, 21, 58, 63, 50, 19, 31, 51, 69, 22, 47, 20, 65, 3, 15, 50, 5, 36, 34, 56, 11, 69, 54, 12, 17, 26, 64, 35, 56, 17, 10, 58, 41, 53, 8, 68, 6, 15, 30, 46, 10, 12, 64, 70, 4, 40, 58, 62, 3, 55, 5, 65, 13, 4, 0, 38, 42, 46, 37, 25, 53, 54, 37, 59, 23, 39, 66, 5, 52, 41, 49, 50, 70, 27, 1, 32, 68, 6, 67, 34, 0, 39, 29, 73, 63, 53, 47, 51, 37, 65, 35, 22, 17, 71, 63, 25, 59, 1, 30, 66, 52, 1, 11, 71, 17, 22, 51, 44, 30, 32, 21, 70, 59, 5, 13, 63, 68, 45, 49, 7, 62, 32, 48, 2, 62, 49, 38, 10, 70, 57, 7, 60, 61, 59, 54, 45, 52, 56, 73, 20, 16, 4, 37, 2, 61, 4, 35, 32, 32, 32, 62, 34, 71, 35, 37, 55, 72, 53, 57, 68, 63, 62, 19, 59, 24, 65, 15, 9, 31, 33, 11, 9, 47, 23, 45, 66, 70, 69, 62, 5, 31, 22, 52, 8, 24, 41, 62, 36, 18, 41, 56, 42, 9, 21, 4, 55, 68, 56, 11, 56, 29, 68, 20, 14, 65, 34, 13, 61, 24, 18, 31, 68, 20, 5, 69, 36, 68, 30, 36, 36, 60, 41, 28, 61, 12, 29, 23, 57, 36, 20, 52, 19, 20, 36, 50, 10, 63, 54, 67, 55, 58, 4, 33, 18, 0, 41, 68, 7, 50, 17, 8, 6, 1, 33, 5, 68, 13, 22, 33, 26, 64, 26, 33, 67, 32, 69, 26, 17]}
{2026: 'TTAATTC', 2059: 'AATTTAT', 1991: 'ATTATTA', 2020: 'ACTAATT', 2001: 'TTTATTT', 2039: 'TAATATA', 2006: 'TTAATAT', 2090: 'ATATATA', 2057: 'ATTAATT', 1985: 'GATTTTT', 2049: 'TTTTTTA', 2206: 'ATTTTCA', 2056: 'ATTTTTT', 2042: 'AAAATTA', 2094: 'TTATATT', 1942: 'TATAAAT', 1999: 'TATTTTA', 2125: 'TTAATTA', 2074: 'ATATTTA', 2031: 'TATCATT', 2088: 'ATAAATT', 2037: 'TCTGAAA', 2084: 'ATTATAA', 2137: 'AATTGTT', 1967: 'AAATAAT', 2116: 'AAATTCT', 2114: 'ATTTTCT', 2027: 'ATATCTT', 2123: 'ATATCTT', 2096: 'TTTAATA', 1881: 'TTAAATA', 1978: 'ATTTAAT', 2141: 'TATTAAA', 2100: 'TTCTATA', 1957: 'ATACTTC', 2111: 'TATATTT', 1936: 'TAATATT', 2138: 'ATTTTCA', 2065: 'AAAATTT', 2112: 'ATTTAAT', 2072: 'ATATAAT', 2194: 'TTAATAT', 2036: 'TATTAAT', 2063: 'TTTTTTA', 2164: 'TTATTTT', 1904: 'TTTAATT'}

Afficher les top motifs (leurs sequence consensus) de meilleurs scores.

In [ ]:
def printTopMotifsScore(consensus, top):

    motifsSort = sorted(consensus.keys())
    motifsSort.reverse()
    for i in range(top):
      # try:
        print (consensus[motifsSort[i]])
      # except IndexError:
      #     pass

top = 10
printTopMotifsScore(consensus, top)

Réponse:

TTTCTCT
AATATTT
AATATAA
ATATTTT
AATTTCT
AATTATT
AATTATA
TATGAAT
AGAAGTT
TATTTTT

13\. Nous allons implémenter l'algorithme ``randomProjection``. D'abord, faites la fonction `getRandomFixePositions` pour générer une projection de k à n, voir un exemple dans les slides de cours. Faire aussi la fonction `generateKey` qui extrait les caractères du motif puis génère une cle qui représente la projection.

In [ ]:
def getRandomFixePositions(k, n):
    """
    Genere une projection de k vers n
    entrée k: nombre de positions du motif original
    entrée n: nombre de positions choisi pour la projection 
    sortie projection: liste de positions choisi aléatoirement
    """    
    return random.sample(range(k), n)

lR = getRandomFixePositions(7, 4)
print (lR)

def generateKey(positions, motif):
    """
    extrait les caractères du motif et génère la cle de la projection
    entrée positions : liste de positions qui represent la projection
    entrée motif : motif de taille k
    sortie cle : cle de la projection
    """
    return "".join(motif[positions[i]] for i in range(len(positions)))

generateKey(lR, "01234567")

Réponse:

[3, 6, 4, 2]


14\. Implémenter l'algorithme ``randomProjection``.

In [ ]:

    def isRepeatN(seq, repeatMax = 5):
    repeatDict = {nu: 0 for nu in nuc}
    for nu in seq:
        repeatDict[nu] += 1
        if repeatDict[nu] >= repeatMax:
            return True
    return False

def checkRepeat2(seq, repeat2Max = 5):
    repeat2s = {(n1+n2)*(repeat2Max//2)+n1*(repeat2Max % 2) for n1 in nuc for n2 in nuc if n1!=n2}
    for repeat2 in repeat2s:
        if seq.find(repeat2) != -1:
            return True
    return False

def randomProjection(k, n, sequences, removeLowComplexe = False):
    """
    Implémente l'algorithme randomProjection
    entrée k : nombre de positions du motif
    entrée n : nombre de positions de la projection 
    entrée sequences : matrice de dimension txn contenant les séquences 
    sortie motifs : dictionaire, cle = projection, valeur= frequence
    sortie motifsSeq:  dictionaire, cle = projection, valeur= original motif
    """
    
    motifs  = dict(); motifsSeq  = dict()
    posFixed = getRandomFixePositions(k, n)
    not_valid_motif = set()
    for i in range(len(sequences)):
        seq = sequences[i]
        for j in range(len(seq)-k+1):
            motif =  seq[j:j+k]

key = generateKey(posFixed, motif)
            if key not in motifs.keys():
                if key not in not_valid_motif:
                    if removeLowComplexe:
                        if not checkRepeat2(motif) and not isRepeatN(motif):
                            motifs[key] = 1
                            motifsSeq[key] = motif
                        else:
                            not_valid_motif.add(key)
                    else:
                        motifs[key] = 1
                        motifsSeq[key] = motif
            else:
                motifs[key] += 1
    return motifs, motifsSeq


adnTest = ['TTAACGGCAC', 'GCTCACGCCA', 'TACCGGCCGT']
motifsProj, motifsSeq = randomProjection(7, 4, adnTest)
print (motifsProj)
print (motifsSeq)

def sortPS(motifsProj, motifsSeq, rev = True):
    return sorted([(motifsProj[k], motifsSeq[k], k) for k in motifsProj.keys()], key=lambda item:item[0], reverse = rev)
    sortPS(motifsProj, motifsSeq)

    Réponse:

    [4, 6, 0, 3]
{'CCCG': 1}
{'CCCG': 'CGGCCGT'}

15\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour augmenter les chances de le trouver. 

reponse: 

In [ ]:
Réponse à la question 15:
Oui possibilité de trouver le motif implémenté, sauf que ce motif n'aura pas la plus 
grande fréquence, en plus il aura souvent une variation. cependant, si on exécute plusieurs fois le code, on augmente la chance de le trouvé

nuc = ('A', 'C', 'G', 'T')

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence

mv3 = implantMotifVar(k, v, t, n)
print("sequences:", mv3)

projMv3, seqMv3 = randomProjection(k, 3, mv3)
print(sortPS(projMv3, seqMv3))

Réponse:

sequences: ['TTCCGATTCC', 'GCAGACCATT', 'GAGGCAATCC']
[(1, 'AATCC', 'AAC')]

16\. Implémenter la version itérative de l’algorithme ``randomProjection``. 

In [ ]:
def randomProjIt(sequences, k, v, nuc, It=50, removeLowComplexe=False):
    """
    Implémente l'algorithme randomProjection version iteractive
    entrée sequences : matrice de dimension txn contenant les séquences 
    entrée k : nombre de positions du motif
    entrée v : nombre de positions de la projection 
    entrée nuc : alphabet
    entrée It: nombre d'iterations
    sortie score : meilleur score
    sortie motifs :  liste de motifs associés au meilleur score
    """
    
    motifs = []; scores = []
    for i in range(It):
        motifsProj, motifsSeq = randomProjection(k, v, sequences, removeLowComplexe)
        bestScore, motif, bucket = sortPS(motifsProj, motifsSeq)[0]
        scores.append(bestScore)
        motifs.append(motif)

    return max(scores), motifs[scores.index(max(scores))] 

score, seqsMotif = randomProjIt(adn, 7, 4, nuc, 100)

print ("score: ",score,"sequence: " ,seqsMotif)

Réponse:

[6, 1, 3, 0]
score:  1 sequence:  CTGCTAG



17\. Tester l'algorithme  ``randomProjection`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats. Puis générér le LOGO du motif trouvé.

In [ ]:
score, seqsMotif = randomProjIt(sequencesChip, 7, 4, nuc, 100)
print ("score: ",score,"sequence: " ,seqsMotif)

Réponse:

[1, 5, 0, 3]
score:  1 sequence:  TATGGCA


18\. Vous avez sans doute remarqué que les algos implémentés trouvent, suivant des motifs peu complexes, réviser les implémentations pour régler/diminuer ce problème.

In [ ]:
def reversecompl(seq):
    ""
    complement={'A':'T','C':'G','G':'C','T':'A'}
    res =''
    for index in range(len(seq)):
        res+=complement[seq[index].Upper()]
    return res[::-1]

score, seqsMotif = randomProjIt(sequencesChip, 7, 4, nuc, 100, True)
print ("score: ",score,"sequence: " ,seqsMotif)
complement ={'A':'T','C':'G','C':'T','T':'A'}
sequencesChipCom=[]
for i in sequencesChip:
    sequencesChipCom.append(reversecompl(i))
score, seqsMotif = randomProjIt(sequencesChipCom, 7, 4, nuc, 100, True)
print ("score: ",score,"sequence: " ,seqsMotif)

Réponse:

[0, 5, 4, 6]
score:  2 sequence: GGACAAA
score:  513 sequence: TGTTATA
score:  513 sequence: TATAACA
